# Quadratic Assignment Problem mit ILS
### Beschreibung des Planungsproblems & Inputdaten

Planungsproblem

Beim QAP-Problem geht es darum, n Anlagen rationell auf n Standorte(Stationen) zu verteilen, so dass die Gesamttransportleistung minimiert wird. Dabei ist die Gesamttransportleistung gleich der Summe aus der Entfernung multipliziert mit der Menge der Transporte. Es wird davon ausgegangen, dass jeder Standort identisch ist, d. h. jede Anlage kann an einem beliebigen Standort platziert werden. Es gibt jedoch eine Situation zu berücksichtigen, in der die Anlage i und die Anlage j an zwei verschiedenen Standorten platziert sind, was dazu führen würde, dass der Abstand zwischen Anlage i und Anlage j geringer ist als der Mindestabstand, der zwischen ihnen eingehalten werden sollte.



Inputdaten
Das Format der Inputdata ist json-Datei, wobei jede Datei die folgenden 4 key-value-paare in einem  Format wie Dictionary enthält.
- "Size": gibt an, wie viele Anlagen sowie Stationen insgesamt an dieser Datei beteiligt sind, der Typ ist int
- "A": drückt die Transportmengen zwischen verschiedenen Anlagen (i und j) in Form einer Matrix aus
- "D": gibt den Abstand zwischen den für die Permutation verfügbaren Standorten (k und l) in Form einer Matrix an
- "M": gibt in Matrixform den Mindestabstand an, der zwischen den verschiedenen Anlagen (i und j) eingehalten werden sollte

Da die Matrizen A und M die Transportmengen und die Mindestentfernungen zwischen den verschiedenen Anlagen darstellen, während die Matrix D die Entfernungen zwischen den verschiedenen Standorten repräsentiert. Wegen der Einfachheit des Codes werde ich daher bei der Suche nach einer Lösung die D-Matrix neu anordnen, d. h. jede Station einem Gerät zuordnen, so dass die neu angeordnete D-Matrix tatsächlich dem Abstand zwischen verschiedenen Anlagen und nicht mehr dem Abstand zwischen Standorten entspricht.

Darüber hinaus wird die Matrix M verwendet, um festzustellen, ob die aktuelle Lösung zulässig ist. Wenn die Anlagen gemäß der aktuellen Lösung(Permutation) angeordnet sind, muss der Abstand zwischen den Anlagen i und j dem durch die M-Matrix vorgegebenen Mindestabstand entsprechen. Andernfalls ist die aktuelle Lösung nicht zulässig.




Die Idee, die meiner Problemlösung zugrunde liegt, ähnelt dem im Seminar vorgestellten Prozess. Zunächst versuche ich, mit Hilfe des ROS-Algorithmus eine Startlösung für jede Instanz zu finden. Nach meinen Ergebnissen liegt diese Startlösung in der Regel deutlich kleiner als der Gesamttransport ohne jegliche Optimierungsmaßnahmen. Ich betrachte die nicht optimierten Ergebnisse als FCFS.

Die Startlösung wird dann als Eingabe für den ILS-Algorithmus verwendet, um die bessere Lösung in der Umgebung der Ausgangslösung zu berechnen.

Es gibt jedoch eine Situation, in der ROS möglicherweise nicht in der Lage ist, eine zulässige Startlösung für die Instanz mit den meisten Geräten (tai25b) zu erzeugen, wenn der Mindestabstand zwischen den Geräten berücksichtigt wird. 

In diesem Fall habe ich jedoch bei meiner Untersuchung des FCFS-Schemas festgestellt, dass die initiale Permutation von tai25b im nicht optimierten Zustand nicht gegen den Mindestabstand verstößt. Daher habe ich diese zulässige Lösung als Eingabe für den ILS-Algorithmus verwendet und eine bessere zulässige Lösung erhalten.



UML Diagramm

<img src="UML.png">


 class InputData
 
Der Zweck dieser Klasse ist es, die json-Datei in ein Wörterbuchformat zu lesen und dann jeden der vier Werte im Wörterbuch: Size, A, D und M, auf jede der vier Attributen der InputData-Klasse zu setzen.

DataLoad() Funktion:  direkt in der Funktion __init__() aufgerufen, um die Datei zu laden und die Attributen beim Erzeugen des Objekts festzulegen

str() Funktion: um direkt auszugeben, wie viele Anlagen gibt es durch print()

Außerdem habe ich bei der Entwicklung des Programms das Attribut path aus Sicherheitsgründen auf privat gesetzt. Dadurch wird sichergestellt, dass die Originaldateien während der Projektbearbeitung nicht von außen zugriffen werden.

In [80]:

import json
import numpy as np

In [81]:
class InputData:
    def __init__(self, path):
        self.__path = path

        self.DataLoad() 

    def DataLoad(self):
        with open(self.__path, "r") as inputFile:
            inputData = json.load(inputFile)

        self.Size = inputData["Size"]

        # np.array()，
        # Die Verwendung des Array-Typs um intuitiver zu sein. 
        # Es ist auch möglich, Berechnungen durchzuführen, für die der Listentyp nicht geeignet ist
        self.Transport_Matrix = np.array(inputData['A']) # Transportmengenmatrix, von Anlage i nach Anlage j
        self.Distance_Matrix = np.array(inputData['D'])  # Entfernungsmatrix zwischen den Stationen, von k bis l
        self.Mind_Entfernung = np.array(inputData['M'])  # Zwischen den Anlagen i und j ist der Mindestabstand einzuhalten

    def __str__(self):
        return f'Quadratic Assignment Problem von {self.Size} Anlagen. '




In [82]:
# Mit Hilfe der InputData-Klasse werden die vier json-Dateien gelesen
data = InputData('scr12.json')
data_15 = InputData("scr15.json")
data_20 = InputData("scr20.json")
data_25 = InputData("tai25b.json")

In [83]:
'''Ich habe hier drei Funktionen definiert, um zu prüfen, ob die Matrizen A, D und M symmetrisch sind. 
Dies ist wichtig, um die Merkmale der Daten in einem frühen Stadium des Projekts zu verstehen.'''

def checkSymA(data):
    result = 0
    # Die Anzahl der Zeilen und Spalten der Matrix ist gleich
    for i in range(data.Size):
        for j in range(data.Size):
            if data.Transport_Matrix[i][j] != data.Transport_Matrix[j][i]:
                result += 1
                
            else:
                result += 0

    return result

def checkSymD(data):
    result = 0
    for i in range(data.Size):
        for j in range(data.Size):
            if data.Distance_Matrix[i][j] != data.Distance_Matrix[j][i]:
                result += 1
                
            else:
                result += 0

    return result

def checkSymM(data):
    result = 0
    for i in range(data.Size):
        for j in range(data.Size):
            if data.Mind_Entfernung[i][j] != data.Mind_Entfernung[j][i]:
                result += 1
                
            else:
                result += 0

    return result

Symmetrie prüfen

Die Ausgabe der drei Zellen unten zeigt, dass die A-Matrix in allen vier Dateien symmetrisch ist.

Bei 25 Geräten sind die D und M Matrizen asymmetrisch. Die Asymmetrie von D wird durch Einbahnstraßen innerhalb des Industriegebiets verursacht, und die Asymmetrie M ist wahrscheinlich auf die unterschiedlichen Anforderungen der verschiedenen Anlagen an den mindesten Abstand zurückzuführen.

Im Falle von scr12, scr15 und scr20 sind alle drei Matrizen symmetrisch

In [84]:
print(checkSymA(data))
print(checkSymA(data_15))
print(checkSymA(data_20))
print(checkSymA(data_25))

0
0
0
0


In [85]:
print(checkSymD(data))
print(checkSymD(data_15))
print(checkSymD(data_20))
print(checkSymD(data_25))

0
0
0
498


In [86]:
print(checkSymM(data))
print(checkSymM(data_15))
print(checkSymM(data_20))
print(checkSymM(data_25))

0
0
0
36


class OutputData

Der Zweck dieser Klasse ist die Vorverarbeitung von InputData, um die gesamte Transportleistung ohne Optimierungsmaßnahmen (Ich sehe die Lösung ohne Optimierung als FCFS) auszugeben. Dies ermöglicht einen Vergleich mit zukünftigen optimierten Lösungen(durch ROS Heuristik und durch ILS) und spiegelt den Notwendigkeit von Prozessoptimierung wider. outputData hat zusätzlich zu den vier Attributen der Klasse InputData die folgenden drei Attributen: InitialPermutation, WithMinimumDistance gesamtTransportLeistung. 

- InitialPermutation: 
- InitialPermutation bezieht sich auf die Seriennummer aller Standorten in aufsteigender Reihenfolge, beginnend mit 0. InitialPermutation ist eine Anordnung von Stationen, die so interpretiert werden kann, dass die Station einer Anlage geschenkt wird. **Alle späteren Permutationen beziehen sich auf die Anordnung von Standorten, nicht auf Anlagen**.

- WithMinimumDistance: 
- WithMinimumDistance ist ein boolescher Wert, der angibt, ob wir die Matrix M(Mindestabstand) bei der Konstruktion eines OutputData-Objekts berücksichtigen sollen oder nicht. Dieses Attribut wird auch in anderen Klassen erscheinen. Das Attribut WithMinimumDistance ermöglicht eine logische Unterscheidung zwischen der Berücksichtigung und Nichtberücksichtigung der M-Matrix, um unnötige Code-Laufzeit zu reduzieren.
Bei der Berechnung des Gesamttransports kann dieses Attribut beispielsweise dazu verwendet werden, um festzustellen, ob die aktuelle Permutation die M-Matrix berücksichtigt, und somit ein Urteil über die Zulässigkeit der Permutation zu fällen.

- gesamtTransportLeistung: 
- Dabei wird die gesamtTransportLeistung durch die Funktion CalculateInitialTransport() berechnet. Da in dieser Klasse nur die ursprüngliche Permutation betrachtet wird, wird die Umordnung der D-Matrix ignoriert. Das heißt, alle Anlagen werden an der gleichen Stelle wie die Seriennummer des jeweiligen Stationen platziert. Ich habe diese Funktion für zwei Zwecke eingerichtet: Erstens wollte ich den gesamten nicht optimierten Transport für jede Instanz ohne Berücksichtigung der M-Matrix ermitteln und mit den Ergebnissen vergleichen, die durch verschiedene Algorithmen optimiert wurden. Zweitens wollte ich eine allgemeine Vorstellung davon bekommen, ob die ursprüngliche Lösung (FCFS) eine zulässige Lösung ergeben könnte, wenn die M-Matrix berücksichtigt wird.



In [87]:
import copy
class OutputData:
    def __init__(self, inputData, withMinimumDistance = False):
        self.InputData = inputData
        self.Size = self.InputData.Size
        self.A = self.InputData.Transport_Matrix
        self.D = self.InputData.Distance_Matrix
        self.M = self.InputData.Mind_Entfernung

        # Die Ausgabe hier ist die ursprüngliche Permutation, d.h. fcfs, so dass es nicht notwendig ist, die Ausrichtung von Station hier zuerst zu berücksichtigen
        self.InitialPermutation = list(range(self.InputData.Size)) 
        self.WithMinimumDistance = withMinimumDistance
        self.gesamtTransportLeistung = self.CalculateInitialTransport()
        
    def CalculateInitialTransport(self):
        if self.WithMinimumDistance == False:
            return (self.A * self.D).sum()
        else:

            if (self.D >= self.M).all() == False:
                return 999999999
            else:
                return (self.A * self.D).sum()

    def __str__(self):
        return f'OutputData: \nQuadratic Assignment Problem von {self.InputData.Size} Anlagen, mit einer initiallen Transportleistung von {self.gesamtTransportLeistung}.'

OutputData-Objekte erzeugen

Konstruieren Objekte der Klasse OutputData aus Objekten der Klasse InputData und geben die GesamtTransportLeistungen aus, die sich ergeben würde, wenn wir keine Optimierung durchführen würden, d. h. wenn wir die Anlagen nach FCFS anordnen würden.



In [88]:
# Erzeugen von OutputData-Objekte, Parameter sind InputData-Objekte
output_12 = OutputData(data)        # scr12 obne Mindestabstand
output_15 = OutputData(data_15)     # scr15 obne Mindestabstand
output_20 = OutputData(data_20)     # scr20 obne Mindestabstand
output_25 = OutputData(data_25)     # tai25b obne Mindestabstand

print(output_12.CalculateInitialTransport())
print(output_15.CalculateInitialTransport())
print(output_20.CalculateInitialTransport())
print(output_25.CalculateInitialTransport())



50116
77278
199318
868229041


In [89]:
# Erzeugen von OutputData-Objekte für alle Instanzen, Parameter True zeigt die Berücksichtigung von M Matrix 
output_12_m = OutputData(data, True)
output_15_m = OutputData(data_15, True)
output_20_m = OutputData(data_20, True)
output_25_m = OutputData(data_25, True)

# Die drei sind unzulässig

print(output_12_m.CalculateInitialTransport())
print(output_15_m.CalculateInitialTransport())
print(output_20_m.CalculateInitialTransport())

# Das ist zulässig!
print(output_25_m.CalculateInitialTransport())  

999999999
999999999
999999999
868229041


Ich habe mir hier eine Logik ausgedacht, um die Durchführbarkeit der Lösung zu bestimmen, die später auf die Funktion CheckAvailable in der Klasse Solution angewendet werden kann.

Die Ausgabe ist False, was bedeutet, dass diese Lösung nicht durchführbar ist. Wir können auch die beiden Matrizen in der Ausgabe untersuchen und es ist leicht zu sehen, dass es tatsächlich Fälle gibt, in denen die Elemente in der D-Matrix kleiner sind als die entsprechenden Elemente in der M-Matrix. Dies zeigt, dass meine Logik richtig ist.

Nachdem ich die Klasse Solution definiert habe, führe ich die gleiche Operation auch mit der Funktion CheckAvailable durch, mit genau dem gleichen Ergebnis.

In [90]:
# Distance Matrix von scr12:
print(output_12_m.D)

# Mindestabstand Matrix von scr12:
print(output_12_m.M)

# Nur wenn alle Elemente in D größer gleich entsprechende Elemente in M ist die Lösung zulässig, unter Berücksichtigung der M Matrix
print((output_12_m.D >= output_12_m.M).all())

[[0 1 2 3 1 2 3 4 2 3 4 5]
 [1 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 1 2]
 [4 3 2 3 3 2 1 2 2 1 0 1]
 [5 4 3 2 4 3 2 1 3 2 1 0]]
[[0 2 0 0 0 0 0 0 0 0 2 0]
 [2 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 2 0]
 [2 0 2 0 0 0 0 0 0 2 0 2]
 [0 0 2 0 0 0 0 0 0 0 2 0]]
False


# class Solution
Ich nehme ein Objekt der Klasse OutputData und eine Permutation als Argumente für den Konstruktor der Klasse Solution. Die Klasse Solution muss die D-Matrix entsprechend dem Parameter "Permutation" mit Hilfe der Funktion "ArrangeDistanceMatrix()" neu anordnen. 

Ich möchte in der Lage sein, die Permutation eines Solution-Objekts jederzeit zu ändern, so dass es einen anderen Gesamttransport ausgibt, also brauche ich die Funktion setPermutation, um das Attribut 'Permutation' zu ändern. Nach der Änderung von Permutation habe ich auch anschließend self.ArrangeDistanceMatrix() aufgerufen, um die D-Matrix neu anzuordnen.

Es ist jedoch anzumerken, dass nicht alle Permutationen zulässig sind, wenn man die M-Matrix betrachtet. Ich habe daher den Parameter Available mit dem Default-Wert True hinzugefügt, um anzuzeigen, dass eine bestimmte Permutation eine ist, die den durch die M-Matrix festgelegten kürzesten Abstand nicht verletzt. Dies ist natürlich nur die Default-Situation. Außerdem muss ich mit der CheckAvailable-Funktion eine Zulässigkeitsprüfung der aktuellen Permutation der Klasse Solution durchführen. Sobald ein Verstoß gegen die von M-Matrix festgestellten Abstände auftreten würde, wird das Attribut Available auf False geändert. 

Wenn ich das Objekt der Klasse Solution ausdruke, möchte ich folgende Ausgabe sehen: Die Permutation [...] führt zu einer Transportleistung von..., was die Funktion str ist. Standardmäßig ist der Wert des Attributs TotalTransport -1 und der TotalTransport wird in der Solution-Klasse nicht berechnet. Der Berechnungsprozess ist in der EvaluationLogic-Klasse untergebracht, so dass jedes Mal, wenn ein Solution-Objekt erstellt wird, der TotalTransport mit Hilfe der EvaluationLogic-Klasse berechnet wird.




In [91]:
class Solution:

    def __init__(self, outputData, permutation):
        self.OutputData = outputData
        self.Permutation = permutation
        self.TransportMatrix = self.OutputData.A
        self.MindDistanceMatrix = self.OutputData.M
        self.setPermutation(permutation)
        self.TotalTransport = -1  

        self.Available = True

    def __str__(self):
        return f'Die Permutation {self.Permutation} führt zu einer Transportleistung von {self.TotalTransport}'

    def setPermutation(self, permutation):
        self.Permutation = permutation
        self.ArrangeDistanceMatrix()

    # Ordnen die Zeilen und Spalten der D-Matrix entsprechend der Permutation neu an, 
    # so dass es sinnvoll ist, die A- und D-Matrizen miteinander zu multiplizieren
    def ArrangeDistanceMatrix(self):
        arrangedDMatrix = self.OutputData.D[self.Permutation, :]
        arrangedDMatrix = arrangedDMatrix[:, self.Permutation]

        self.ArrangedDMatrix = arrangedDMatrix

        return arrangedDMatrix

    
    def CheckAvailable(self):

        if (self.ArrangedDMatrix >= self.MindDistanceMatrix).all():
            self.Available = True
        else:
            self.Available = False
                

Ich habe drei Solution-Objekte mit output_12 und drei verschiedenen Permutationen in jeder der nächsten drei Zellen konstruiert und versucht zu überprüfen, ob die aktuelle Permutation unter Berücksichtigung der M-Matrix zulässig ist. Leider konnte keine der drei Permutationen die Mindestabstände erfüllen. Daher vermute ich, dass die optimale Lösung später im Optimierungsprozess schwer zu finden sein könnte, wenn die M-Matrix berücksichtigt werden soll. Aber keine Sorge, denn ich denke, dass sich durch mehrere Iterationen mehrere zulässige Lösungen finden lassen.

In [92]:
sol_1 = Solution(output_12, list(range(12))) 
print(sol_1.ArrangedDMatrix)
print(sol_1.MindDistanceMatrix)

sol_1.CheckAvailable()
print(sol_1.Available)

[[0 1 2 3 1 2 3 4 2 3 4 5]
 [1 0 1 2 2 1 2 3 3 2 3 4]
 [2 1 0 1 3 2 1 2 4 3 2 3]
 [3 2 1 0 4 3 2 1 5 4 3 2]
 [1 2 3 4 0 1 2 3 1 2 3 4]
 [2 1 2 3 1 0 1 2 2 1 2 3]
 [3 2 1 2 2 1 0 1 3 2 1 2]
 [4 3 2 1 3 2 1 0 4 3 2 1]
 [2 3 4 5 1 2 3 4 0 1 2 3]
 [3 2 3 4 2 1 2 3 1 0 1 2]
 [4 3 2 3 3 2 1 2 2 1 0 1]
 [5 4 3 2 4 3 2 1 3 2 1 0]]
[[0 2 0 0 0 0 0 0 0 0 2 0]
 [2 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 2 0]
 [2 0 2 0 0 0 0 0 0 2 0 2]
 [0 0 2 0 0 0 0 0 0 0 2 0]]
False


In [93]:
print((sol_1.ArrangedDMatrix >= sol_1.MindDistanceMatrix).all())

False


In [94]:
sol_2 = Solution(output_12, [0,11,1,10,2,9,3,8,4,7,5,6])
print(sol_2.ArrangedDMatrix)
print(sol_2.MindDistanceMatrix)

sol_2.CheckAvailable()
print(sol_2.Available)

[[0 5 1 4 2 3 3 2 1 4 2 3]
 [5 0 4 1 3 2 2 3 4 1 3 2]
 [1 4 0 3 1 2 2 3 2 3 1 2]
 [4 1 3 0 2 1 3 2 3 2 2 1]
 [2 3 1 2 0 3 1 4 3 2 2 1]
 [3 2 2 1 3 0 4 1 2 3 1 2]
 [3 2 2 3 1 4 0 5 4 1 3 2]
 [2 3 3 2 4 1 5 0 1 4 2 3]
 [1 4 2 3 3 2 4 1 0 3 1 2]
 [4 1 3 2 2 3 1 4 3 0 2 1]
 [2 3 1 2 2 1 3 2 1 2 0 1]
 [3 2 2 1 1 2 2 3 2 1 1 0]]
[[0 2 0 0 0 0 0 0 0 0 2 0]
 [2 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 2 0]
 [2 0 2 0 0 0 0 0 0 2 0 2]
 [0 0 2 0 0 0 0 0 0 0 2 0]]
False


In [95]:
sol_3 = Solution(output_12, [0,2,4,6,8,10,11,9,7,5,3,1])
print(sol_3.ArrangedDMatrix)
print(sol_3.MindDistanceMatrix)
sol_3.CheckAvailable()
print(sol_3.Available)

[[0 2 1 3 2 4 5 3 4 2 3 1]
 [2 0 3 1 4 2 3 3 2 2 1 1]
 [1 3 0 2 1 3 4 2 3 1 4 2]
 [3 1 2 0 3 1 2 2 1 1 2 2]
 [2 4 1 3 0 2 3 1 4 2 5 3]
 [4 2 3 1 2 0 1 1 2 2 3 3]
 [5 3 4 2 3 1 0 2 1 3 2 4]
 [3 3 2 2 1 1 2 0 3 1 4 2]
 [4 2 3 1 4 2 1 3 0 2 1 3]
 [2 2 1 1 2 2 3 1 2 0 3 1]
 [3 1 4 2 5 3 2 4 1 3 0 2]
 [1 1 2 2 3 3 4 2 3 1 2 0]]
[[0 2 0 0 0 0 0 0 0 0 2 0]
 [2 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 2 0]
 [2 0 2 0 0 0 0 0 0 2 0 2]
 [0 0 2 0 0 0 0 0 0 0 2 0]]
False


### class SolutionPool

Die Klasse SolutionPool ist ein Container, in dem Objekte der Klasse Solution gespeichert werden. In diesem Container werden die während des Optimierungsprozesses erzeugten Lösungen ("Solution" Objekte) gespeichert, so dass wir die Funktion AddSolution benötigen, um neue Lösungen hinzuzufügen.

Darüber hinaus muss der SolutionPool die bisher beste Lösung im Container exportieren. Das heißt, die Funktion GetLowestTransportSol, deren Rückgabewert ein Objekt der Klasse Solution ist, so dass diese Ausgabe die Attribute und Mitgliedsfunktionen der Klasse Solution haben sollte.



In [96]:
class SolutionPool:
    def __init__(self):
        self.Solutions = []

    def AddSolution(self, newSolution):
        self.Solutions.append(newSolution)

    def GetLowestTransportSol(self):
        # Sortieren der Liste der self.Solutions in aufsteigender Reihenfolge nach dem Gesamttransport jeder Lösung 
        # und return das erste Element in der neu sortierten Liste, d. h. der aktuellen optimalen Lösung
        self.Solutions.sort(key = lambda solution: solution.TotalTransport) 

        return self.Solutions[0]

### class EvaluationLogic

Der Zweck dieser Klasse ist es, die gesamte Transportleistung getrennt zu berechnen, wenn die kürzeste Entfernung berücksichtigt wird und wenn die kürzeste Entfernung nicht berücksichtigt wird. Der Mindestabstand, der zwischen den Anlagen eingehalten werden soll, wird in der M-Matrix gespeichert.

Gleichzeitig kann die Klasse "EvaluationLogic" auch das Attribut "TotalTransport" der Objekte der Klasse "Solution" ändern. Der Default-Wert -1 von Solution Objekten wird erst nach der EvaluationLogic geändert.

Ich habe nicht nur den Default-Wert des Attributs von Solution geändert, sondern auch den Rückgabewert in der Funktion DefineTotalTransport festgelegt, so dass Zuweisungen über diese Funktion vorgenommen werden können.

Bei der DefineTotalTransport-Funktion wird zunächst entschieden, ob die kürzeste Abstände berücksichtigt werden sollen. Wenn die kürzeste Entfernung nicht berücksichtigt wird(withMinimumDistance = False), werden die TransportMatrix und die ArrangedDMatrix direkt multipliziert und summiert. Wird der kürzeste Weg betrachtet(withMinimumDistance = True), muss die Zulässigkeit der aktuellen Lösung durch CheckAvailable-Funktion geprüft werden. Wenn die aktuelle Lösung zulässig ist, wird der TotalTransport auf die gleiche Weise berechnet. Stattdessen habe ich den Wert für den TotalTransport auf 999999999 gesetzt. Diese Zahl ist groß genug und besteht aus neun 9en. Wenn die Gesamttransportleistung diese Zahl erreicht, bedeutet dies, dass die aktuelle Lösung nicht machbar ist.

In [97]:
class EvaluationLogic:
    def __init__(self, withMinimumDistance = False):
        # self.InputData = inputData
        self.TotalTransport = None
        self.WithMinimumDistance = withMinimumDistance

# Diese Funktion wird in der Solver-Klasse aufgerufen und gibt das Ergebnis sowohl mit als auch ohne MinimumDistance aus
    def DefineTotalTransport(self, currentSolution):
        if self.WithMinimumDistance == False:
            totalTransport = (currentSolution.TransportMatrix * currentSolution.ArrangedDMatrix).sum()
            
        else:
            currentSolution.CheckAvailable()
            if currentSolution.Available == True:
                totalTransport = (currentSolution.TransportMatrix * currentSolution.ArrangedDMatrix).sum()
            else:
                totalTransport = 999999999
        
        currentSolution.TotalTransport = totalTransport

        return totalTransport


Hier werte ich die drei vorhergehenden Solution-Objekte mit el bzw. el2 aus und gebe den TotalTransport für die drei Solutions in zwei Fällen aus. Ich stelle fest, dass bei der Betrachtung des Mindestabstands alle drei Lösungen 9999999999 ergeben, was auch mit den vorherigen False-Ergebnissen übereinstimmt, die ich zuvor über die CheckAvailable-Funktion ausgegeben habe. Meine Logik ist also richtig.


# Musterlösung

In [ ]:
# 把最优结果的排列生成Solution对象，然后输出最优解，跟https://www.opt.math.tugraz.at/qaplib/inst.html
# 进行对比，发现结果是一样的！！！开心！！！这证明我的运算过程是正确的，唯一需要优化的地方是ILS算法
bestSolution_12 = Solution(output_12, [7,5,2,1,9,0,4,8,3,6,11,10])
EvaluationLogic().DefineTotalTransport(bestSolution_12)
print(bestSolution_12)

bestSolution_15 = Solution(output_15, [14,6,10,7,0,3,2,1,11,5,12,4,13,9,8])
EvaluationLogic().DefineTotalTransport(bestSolution_15)
print(bestSolution_15)

bestSolution_20 = Solution(output_20, [19,6,11,5,3,7,2,1,13,10,17,8,18,14,15,16,12,4,9,0])
EvaluationLogic().DefineTotalTransport(bestSolution_20)
print(bestSolution_20)

bestSolution_25 = Solution(output_25, [3,14,9,8,12,4,24,18,6,2,16,5,17,19,15,1,21,22,7,10,20,23,13,11,0])
EvaluationLogic().DefineTotalTransport(bestSolution_25)
print(bestSolution_25)

Die Permutation [7, 5, 2, 1, 9, 0, 4, 8, 3, 6, 11, 10] führt zu einer Transportleistung von 31410
Die Permutation [14, 6, 10, 7, 0, 3, 2, 1, 11, 5, 12, 4, 13, 9, 8] führt zu einer Transportleistung von 51140
Die Permutation [19, 6, 11, 5, 3, 7, 2, 1, 13, 10, 17, 8, 18, 14, 15, 16, 12, 4, 9, 0] führt zu einer Transportleistung von 110030
Die Permutation [3, 14, 9, 8, 12, 4, 24, 18, 6, 2, 16, 5, 17, 19, 15, 1, 21, 22, 7, 10, 20, 23, 13, 11, 0] führt zu einer Transportleistung von 344355646


In [98]:
# el ist das EvaluationLogic-Objekt ohne Berücksichtigung des Mindestabstands.
el = EvaluationLogic() 

# Mit Berücksichtigung des Mindestabstands.
el2 = EvaluationLogic(True) 

el.DefineTotalTransport(sol_1)
print(sol_1)
el2.DefineTotalTransport(sol_1)
print(sol_1)

el.DefineTotalTransport(sol_2)
print(sol_2)
el2.DefineTotalTransport(sol_2)
print(sol_2)

el.DefineTotalTransport(sol_3)
print(sol_3)
el2.DefineTotalTransport(sol_3)
print(sol_3)

Die Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] führt zu einer Transportleistung von 50116
Die Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] führt zu einer Transportleistung von 999999999
Die Permutation [0, 11, 1, 10, 2, 9, 3, 8, 4, 7, 5, 6] führt zu einer Transportleistung von 63118
Die Permutation [0, 11, 1, 10, 2, 9, 3, 8, 4, 7, 5, 6] führt zu einer Transportleistung von 999999999
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 45322
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 999999999


In [99]:
# die Datentypen von Permutation und TotalTransport zeigen
print(type(sol_1.Permutation))
print(type(sol_1.TotalTransport))

<class 'list'>
<class 'int'>


Nach der Ausführung der Funktion setPermutation wird das Attribut ArrangedDMatrix von sol_22 geändert, das Attribut TotalTransport wird jedoch erst nach der Funktion DefineTotalTransport aktualisiert.

Das heißt, um sowohl die Permutation als auch den Gesamttransport einer Solution im anschließenden Optimierungsalgorithmus zu ändern, muss zuerst die setPermutation-Funktion aufgerufen werden, gefolgt von der DefineTotalTransport-Funktion.

In [100]:
sol_22 = Solution(output_12, [0,11,1,10,2,9,3,8,4,7,5,6])
print(sol_22)

EvaluationLogic().DefineTotalTransport(sol_22) 
print(sol_22)

sol_22.setPermutation([0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1])  
print(sol_22)


EvaluationLogic().DefineTotalTransport(sol_22)    
print(sol_22)

Die Permutation [0, 11, 1, 10, 2, 9, 3, 8, 4, 7, 5, 6] führt zu einer Transportleistung von -1
Die Permutation [0, 11, 1, 10, 2, 9, 3, 8, 4, 7, 5, 6] führt zu einer Transportleistung von 63118
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 63118
Die Permutation [0, 2, 4, 6, 8, 10, 11, 9, 7, 5, 3, 1] führt zu einer Transportleistung von 45322


# class ConstructiveHeuristics
Zu diesem Zeitpunkt wird ein heuristischer Algorithmus benötigt, um eine startSolution zu generieren, die der optimalen Lösung so nahe wie möglich ist. 

Ich habe mich für die ROS-Methode (Random Order of Service) entschieden, d. h. ich setze eine Zufallszahl als Startwert, so dass bei jeder Ausführung der Zelle das gleiche Ergebnis erzielt wird. Anschließend werden mehrere zufällige Permutationen erzeugt, die jeweils eine Anzahl von Elementen enthalten, die der Anzahl der Anlagen entspricht.

Schließlich wird das ROS in der Klasse ConstructiveHeuristics über die Funktion Run aufgerufen, wobei ich den RNG benutzt habe, der insgesamt 40.000 zufällige Permutationen erzeugt.

Ich habe in self.EvaluationLogic den Parameter gesetzt, ob die Mindestabstandsmatrix berücksichtigt werden soll oder nicht. So kann die gesamte Transportleistung für verschiedene Szenarien berechnet werden. Deswegen brauche ich keine Parameter in der Klasse ConstructiveHeuristics zu setzen, um zu zeigen, ob M Matrix berücksichtigt werden soll.

Ich habe jedoch festgestellt, dass sich die Startlösung mit ROS recht einfach berechnen lässt, wenn der Mindestabstand zwischen den Anlagen nicht berücksichtigt wird. Bei der Betrachtung des Mindestabstands zwischen den Anlagen konnte jedoch selbst bei der Verwendung von ROS zur Erzeugung von 40000 zufälligen Permutationen keine zulässige Lösung gefunden werden. Also habe ich eine Funktion "FindStartSol_mMA" geschrieben, um eine zulässige Lösung zu finden.

In der FindStartSol_mMA-Funktion wird zunächst ein Urteil gefällt, und wenn die mit der ROS-Funktion erzeugte aktuelle Lösung zulässig ist, wird diese zurückgegeben. Wenn dies nicht möglich ist, verwende ich die Shuffle-Funktion im Zufallsmodul, um die Anordnung der tmpPermutation zu randomisieren. Nach der Änderung der Permutation wird die Funktion CheckAvailable verwendet, um die Zulässigkeit der aktuellen Lösung zu prüfen. Sobald das Attribut Available der aktuellen Lösung True ist, wird der Schleifenkörper abgebrochen und die aktuelle Lösung zurückgegeben.





### Nachdenken über Performanz und Deep Learning
Aber 400000 zufällige Permutationen sind immer noch eine kleine Zahl für tai25b mit 25! verschiedenen Permutationen, und wenn ich den Mindestabstand betrachte, kann ich auf diese Weise nicht zu einer Startlösung kommen.

Der Grund dafür ist, dass es bei der Zufallsauswahl der 25 Standorte insgesamt 25! gibt. Dies ist zweifellos eine große Zahl, und der ROS-Algorithmus hat Schwierigkeiten, in einer endlichen Anzahl von Iterationen eine zulässige Startlösung zu finden. Ich könnte sicherlich die Anzahl der zufälligen Permutationen erhöhen, um ein möglichst breites Spektrum an Permutationen abzudecken. Dies würde jedoch dazu führen, dass das Programm wesentlich länger laufen würde, was der ursprünglichen Absicht der Verwendung des heuristischen Algorithmus zuwiderläuft.

In der realen Produktion kann man den Optimierungsplan flexibel an die Anforderungen der Firma anpassen. Wenn man genug Zeit hat, um auf die perfekte Lösung zu warten, kann man die Anzahl der zufälligen Permutationen beliebig erhöhen. Ich denke, da sich die Leistung der Computer in den letzten 10 Jahren erheblich verbessert hat, könnte man sogar alle möglichen Permutationen durchspielen, um die perfekte Lösung zu finden. Im Gegensatz zu den CPU-Leistungsbeschränkungen von vor mehr als 20 Jahren, wie in "A survey for the quadratic assignment problem" gezeigt, ist der Optimierungsprozess von QAP sehr langsam.

In der Zwischenzeit habe ich eine weitere ungetestete Idee, dass man die Optimierung des QAP-Problems vielleicht mit Hilfe von neuronalen Netzwerkmodellen im Deep Learning erreichen kann. Man könnte verschiedene Modelle für verschiedene Arten von Problemen trainieren.

In [101]:
import numpy
from copy import deepcopy
class ConstructiveHeuristics:
    def __init__(self, evaluationLogic, solutionPool, rng, randomRetries=400000):  
        self.RNG = rng   # 999  # 2022
        self.RandomRetiris = randomRetries     
        self.EvaluationLogic = evaluationLogic  
        self.SolutionPool = solutionPool

    def ROS(self, outputData, iterations, rng):
        # numpy.random.seed(rng)
        tmpSolution = Solution(outputData, outputData.InitialPermutation)  
        bestCmax = numpy.inf


        for i in range(iterations):
            # Generation von zufälligen Permutationen
            tmpPermutation = rng.permutation(len(outputData.D))  
            tmpSolution.setPermutation(tmpPermutation)

            self.EvaluationLogic.DefineTotalTransport(tmpSolution) 

            if (tmpSolution.TotalTransport < bestCmax):
                bestCmax = tmpSolution.TotalTransport
                bestPerm = tmpPermutation
            
        bestSol = Solution(outputData, bestPerm)
        self.EvaluationLogic.DefineTotalTransport(bestSol)

        return bestSol


    def FindStartSol_mMA(self, outPutData):  #  eine Startsolution mit Beruechtigeng von Mindstabstand finden
        tmpSolution = self.ROS(outPutData, self.RandomRetiris, self.RNG)
        
        if tmpSolution.Available == False:
            x = 0
            while x < 30000  :

                permKopie = list(copy.deepcopy(tmpSolution.Permutation))
                self.RNG.shuffle(permKopie)   # shuffle hat keinen Rückgabewert (none)
                tmpSolution.setPermutation(permKopie)
                tmpSolution.CheckAvailable()
                self.EvaluationLogic.DefineTotalTransport(tmpSolution)

                if tmpSolution.Available == True:
                    break
                x += 1
            return tmpSolution

        else:
            return tmpSolution


    def Run(self, outPutData):
        print('Generating an initial solution according to ROS. ')
        solution = None

        # Ohne Berücksichtigung von M Matrix: ROS direkt aufrufen
        if self.EvaluationLogic.WithMinimumDistance == False:
            solution = self.ROS(outPutData, self.RandomRetiris, self.RNG)
        # Mit Berücksichtigung von M Matrix: FindStartSol_mMA aufrufen, in der auch zuerst ROS aufgerufen wird
        else:
            solution = self.FindStartSol_mMA(outPutData)
            
        self.SolutionPool.AddSolution(solution)


Wie bereits erwähnt, konnte ich mit ConstructiveHeuristics eine StartLösung für 12 Maschinen unter Berücksichtigung des Mindestabstands finden. Da die InitialPermutation von scr12 unzulässig ist, FindStartSol_mMA wird aufgerufen.

Dies bestätigte, dass meine Logik bei der Suche nach der Startsolution richtig war. Mein nächster Schritt bestand also darin, die Solver-Klasse zu konstruieren, um den meta-heuristischen Algorithmus mit Hilfe der Solver-Klasse aufzurufen.


In [102]:

construct_with_m = ConstructiveHeuristics(EvaluationLogic(True), SolutionPool(), np.random.default_rng(999))

construct_with_m.Run(output_12)
print(construct_with_m.SolutionPool.GetLowestTransportSol())

Generating an initial solution according to ROS. 
Die Permutation [ 8  6  0  7 10  1  5  9  4  2 11  3] führt zu einer Transportleistung von 35662


### class Solver
Um eine Solver-Klasse zu erstellen, müssen wir in der init-Funktion mit dem Parameter withMinimumDistance(bei Default: False) angeben, um zu zeigen, ob dieses Solver-Objekt die Mindestabstandsmatrix berücksichtigen soll. Dieser Parameter wird dann an EvaluationLogic übergeben.

- Die Klasse Solver hat zwei Aufgaben: 

1. zunächst die InputData mit dem Solver-Objekt einlesen und dann die Mitgliedsfunktion ConstructionPhase aufrufen, um die initiale Lösung zu erzeugen.

2. Zweitens, um die Funktion ImprovePhase aufzurufen, die die Ausgabe der Funktion ConstructionPhase als Eingabe verwendet und die ursprüngliche Lösung iteriert, um sie zu optimieren

Die Funktion Initialize() dient der Initialisierung des Optimierungsalgorithmus und übergibt die Parameter EvaluationLogic, SolutionPool und RNG an den Optimierungsalgorithmus.

Die Funktion RunLocalSearch ruft zunächst die Funktion ConstructionPhase() auf und verwendet dann deren Ausgabe als Eingabe für ImprovementPhase(). In ImprovementPhase verwende ich den Algorithmus Iterated Local Search.

Nach RunLocalSearch können wir die bestmögliche Lösung erhalten.


In [103]:

class Solver:
    def __init__(self, inputData, seed, withMinimumDistance = False): 
        self.InputData = inputData
        self.Seed = seed
        self.RNG = numpy.random.default_rng(seed)
        self.OutputData = OutputData(self.InputData)
        self.WithMinimumDistance = withMinimumDistance
        self.EvaluationLogic = EvaluationLogic(self.WithMinimumDistance)
        self.SolutionPool = SolutionPool()

        self.ConstructiveHeuristic = ConstructiveHeuristics(self.EvaluationLogic, self.SolutionPool, self.RNG)

    def Initialize(self):
        self.OptimizationAlgorithm.Initialize(self.EvaluationLogic, self.SolutionPool, self.RNG)

    # ConstructionPhase liefert die StartLösung, die mit Hilfe des ROS-Algorithmus ermittelt wurde 
    # und als Eingabe für das Näherungsverfahrens verwendet werden kann
    def ConstructionPhase(self):
        self.ConstructiveHeuristic.Run(self.OutputData)
        bestInitialSol = self.SolutionPool.GetLowestTransportSol()
        
        print('Constructive Solution found. ')
        # print(bestInitialSol)

        return bestInitialSol

    def ImprovementPhase(self, startSolution, algorithm):
        algorithm.Initialize(self.EvaluationLogic, self.SolutionPool, self.RNG)
        bestSolution = algorithm.Run(startSolution)  # 这里调用了ILS类中的Run函数，包含了Pertubation

        print('Best found solution. ')
        return bestSolution

    def RunLocalSearch(self, algorithm):
        startSolution = self.ConstructionPhase()
        bestSolution = self.ImprovementPhase(startSolution, algorithm)

        return bestSolution


Die folgenden vier Codezeilen und ihre Ausgabe zeigen, dass der ROS-Algorithmus in der Lage ist, eine Lösung zu erzeugen, die der optimalen Lösung für den Fall von 12 Geräten relativ nahe kommt. Außerdem liegt das mit und ohne Berücksichtigung der Mindestentfernung berechnete TotalTransport relativ nahe beieinander.

In [104]:
# initiale Solution von scr_12 ohne Berücksichtigung von M
solver = Solver(data, 999, False)
print(solver.ConstructionPhase())


# initiale Solution von scr_12 mit Berücksichtigung von M
solver_m = Solver(data, 999, True)
print(solver_m.ConstructionPhase())

Generating an initial solution according to ROS. 
Constructive Solution found. 
Die Permutation [ 9 11  4 10  2  6  7  3  8  1  0  5] führt zu einer Transportleistung von 34242
Generating an initial solution according to ROS. 
Constructive Solution found. 
Die Permutation [ 8  6  0  7 10  1  5  9  4  2 11  3] führt zu einer Transportleistung von 35662


Ich generiere hier eine Startlösung für scr12 ohne Berücksichtigung der M-Matrix, und später werde ich diese Lösung mit Hilfe des Näherungsverfahrens optimieren. um den Grad der Optimierung zu beobachten.

In [105]:

bestInitialSolution = solver.ConstructionPhase()

print(bestInitialSolution)


Generating an initial solution according to ROS. 
Constructive Solution found. 
Die Permutation [10  5  6  9  1  8  4  0  2  7 11  3] führt zu einer Transportleistung von 33560




Erzeugen eines Lösungsobjekts aus der Permutation des optimalen Ergebnisses(https://www.opt.math.tugraz.at/qaplib/inst.html) und Ausgeben der optimalen Lösung.

Meine Ausgabe ist identisch wie optimale Lösung.

Da ich in meinem Code die Permutationen der Standorte (D-Matrix) gemäß dem Attribut Permutation von Solution, und nicht die Permutationen der Anlagen behandelt habe, erhalte ich das gleiche Ergebnis wie die echte optimale Lösung. Was darauf schließen lässt, dass die Permutation in der echten optimalen Lösung auch eine Permutation der Standorte ist (der Index der Liste sind die Anlagen und die Elemente der Liste sind die Standorte).

Die Belegarbeit erfordert, dass die Ausgabe eine Permutation von Anlagen ist (eine Anordnung, bei der verschiedene Anlagen den Standorten 0, 1 ... n-1 zugeordnet sind, d. h. der Index der Liste ist die Position und die Elemente der Liste sind die Anlagen).

Um schließlich ein Ergebnis auszugeben, das den Anforderungen des Lehrstuhls entspricht, habe ich daher eine changePermutation-Funktion definiert, die die Anordnung der Positionen in eine Anordnung der Anlagen umwandelt.

Aber dann habe ich mit dem Lehrer gesprochen und erfahren, dass ich die Permutation der Standorte auch unverändert lassen kann. Ich habe mich aber dennoch entschieden, die Definition dieser changePermutation beizubehalten, weil es für die Menschen einfacher ist, den Prozess der Platzierung der Maschine an verschiedenen Standorten zu verstehen. Ich dachte, dass ich diese Funktion in der Praxis nutzen könnte.

In [106]:
# 把最优结果的排列生成Solution对象，然后输出最优解，跟https://www.opt.math.tugraz.at/qaplib/inst.html
# 进行对比，发现结果是一样的！！！开心！！！这证明我的运算过程是正确的，唯一需要优化的地方是ILS算法
bestSolution_12 = Solution(output_12, [7,5,2,1,9,0,4,8,3,6,11,10])
EvaluationLogic().DefineTotalTransport(bestSolution_12)
print(bestSolution_12)

bestSolution_15 = Solution(output_15, [14,6,10,7,0,3,2,1,11,5,12,4,13,9,8])
EvaluationLogic().DefineTotalTransport(bestSolution_15)
print(bestSolution_15)

bestSolution_20 = Solution(output_20, [19,6,11,5,3,7,2,1,13,10,17,8,18,14,15,16,12,4,9,0])
EvaluationLogic().DefineTotalTransport(bestSolution_20)
print(bestSolution_20)

bestSolution_25 = Solution(output_25, [3,14,9,8,12,4,24,18,6,2,16,5,17,19,15,1,21,22,7,10,20,23,13,11,0])
EvaluationLogic().DefineTotalTransport(bestSolution_25)
print(bestSolution_25)

Die Permutation [7, 5, 2, 1, 9, 0, 4, 8, 3, 6, 11, 10] führt zu einer Transportleistung von 31410
Die Permutation [14, 6, 10, 7, 0, 3, 2, 1, 11, 5, 12, 4, 13, 9, 8] führt zu einer Transportleistung von 51140
Die Permutation [19, 6, 11, 5, 3, 7, 2, 1, 13, 10, 17, 8, 18, 14, 15, 16, 12, 4, 9, 0] führt zu einer Transportleistung von 110030
Die Permutation [3, 14, 9, 8, 12, 4, 24, 18, 6, 2, 16, 5, 17, 19, 15, 1, 21, 22, 7, 10, 20, 23, 13, 11, 0] führt zu einer Transportleistung von 344355646


In [107]:
# Annahme: ap ist die Permutation von Stationen, Anlage 0 auf Station 2 usw.
# Jetzt möchte ich ap in die Permutation von Anlagen wechseln
ap = [2,1,3,0,4] 
perm = []
for i, j in enumerate(ap):
    perm.append((i,j))

sorted_perm = sorted(perm, key = lambda t: t[1])
print(perm)
print(sorted_perm)

new_perm = [x[0] for x in sorted_perm]
print(new_perm)


print('\n\n')

# Dieser Prozess ist reversibel.
ap = [3, 1, 0, 2, 4]
perm = []
for i, j in enumerate(ap):
    perm.append((i,j))

sorted_perm = sorted(perm, key = lambda t: t[1])
print(perm)
print(sorted_perm)

new_perm = [x[0] for x in sorted_perm]
print(new_perm)

[(0, 2), (1, 1), (2, 3), (3, 0), (4, 4)]
[(3, 0), (1, 1), (0, 2), (2, 3), (4, 4)]
[3, 1, 0, 2, 4]



[(0, 3), (1, 1), (2, 0), (3, 2), (4, 4)]
[(2, 0), (1, 1), (3, 2), (0, 3), (4, 4)]
[2, 1, 3, 0, 4]


In [108]:
# Die changePermutation-Funktion dient dazu, die Permutation der Anlagen und die 
# Permutation des Stationen ineinander umzuwandeln. Dieser Prozess ist reversibel.
def changePermutation(perm):
    tmpPerm = []
    for i,j in enumerate(perm):
        tmpPerm.append((i,j))
    sorted_tuple_perm = sorted(tmpPerm, key=lambda t: t[1])
    return [x[0] for x in sorted_tuple_perm]

# Eingabe: [2, 1, 3, 0, 4]ist eine Permutation von Stationen(Elemente der List)
# Anlage 0 auf Station 2, 
# Anlage 1 auf Station 1, 
# Anlage 2 auf Station 3, 
# Anlage 3 auf Station 0, 
# Anlage 4 auf Station 4.

# Ausgabe: [3, 1, 0, 2, 4] ist eine Permutation von Anlagen(Elemente der List)
# Anlage 3 auf Station 0,
# Anlage 1 auf Station 1, 
# Anlage 0 auf Station 2, 
# Anlage 2 auf Station 3, 
# Anlage 4 auf Station 4.
print(changePermutation([2, 1, 3, 0, 4]))

# Dieser Prozess ist reversibel.
print(changePermutation([3, 1, 0, 2, 4]))

[3, 1, 0, 2, 4]
[2, 1, 3, 0, 4]


Wenn ich die Permutation der wahren optimalen Lösung (die Permutation der Standorte) in eine Permutation der Anlagen umwandle, kann ich die optimale Lösung nicht ausgeben. Die Ausgabe des nachstehenden Codeblocks bestätigt meinen Verdacht noch einmal.

In [109]:
bestSolution_12 = Solution(output_12, changePermutation([7,5,2,1,9,0,4,8,3,6,11,10]))
EvaluationLogic().DefineTotalTransport(bestSolution_12)
print(bestSolution_12)

bestSolution_15 = Solution(output_15, changePermutation([14,6,10,7,0,3,2,1,11,5,12,4,13,9,8]))
EvaluationLogic().DefineTotalTransport(bestSolution_15)
print(bestSolution_15)

bestSolution_20 = Solution(output_20, changePermutation([19,6,11,5,3,7,2,1,13,10,17,8,18,14,15,16,12,4,9,0]))
EvaluationLogic().DefineTotalTransport(bestSolution_20)
print(bestSolution_20)

bestSolution_25 = Solution(output_25, changePermutation([3,14,9,8,12,4,24,18,6,2,16,5,17,19,15,1,21,22,7,10,20,23,13,11,0]))
EvaluationLogic().DefineTotalTransport(bestSolution_25)
print(bestSolution_25)

Die Permutation [5, 3, 2, 8, 6, 1, 9, 0, 7, 4, 11, 10] führt zu einer Transportleistung von 80066
Die Permutation [4, 7, 6, 5, 11, 9, 1, 3, 14, 13, 2, 8, 10, 12, 0] führt zu einer Transportleistung von 93506
Die Permutation [19, 7, 6, 4, 17, 3, 1, 5, 11, 18, 9, 2, 16, 8, 13, 14, 15, 10, 12, 0] führt zu einer Transportleistung von 202606
Die Permutation [24, 15, 9, 0, 5, 11, 8, 18, 3, 2, 19, 23, 4, 22, 1, 14, 10, 12, 7, 13, 20, 16, 17, 21, 6] führt zu einer Transportleistung von 905934981


### Optimale Lösungen aus https://www.opt.math.tugraz.at/qaplib/inst.html 

| Datei| n_Anlagen| OPT | Permutation|
|------|----------|------|----------|
|Scr12  |12    |31410 |   (8,6,3,2,10,1,5,9,4,7,12,11)
|Scr15  |15   | 51140  |  (15,7,11,8,1,4,3,2,12,6,13,5,14,10,9)
|Scr20  |20  | 110030   | (20,7,12,6,4,8,3,2,14,11,18,9,19,15,16,17,13,5,10,1)
|Tai25b  |  25 |   344355646 |      (4,15,10,9,13,5,25,19,7,3,17,6,18,20,16,2,22,23,8,11,21,24,14,12,1)

### Meine Lösungen

In den nächsten Codeblöcken gebe ich das nach dem FCFS-Prinzip und der ROS-Methode ermittelte Gesamttransportleistung für die Fälle von 12, 15, 20 bzw. 25 Anlagen aus und vergleiche sie mit der optimalen Lösung.

Es zeigt sich, dass der Einsatz der ROS-Methode die Gesamttransportleistung im Vergleich zu keiner Optimierung(FCFS) bereits deutlich reduzieren kann. Um die optimale Lösung zu erreichen, ist es jedoch notwendig, eine iterative lokale Suche durchzuführen.

1. ohne Berücksichtigung von Mindestabstände


|Datei|FCFS|ROS|ILS|opt|
|---|---|---|---|----|
|scr12|50116|34242|31410   |31410|
|scr15|77278|60208|53114    |51140|
|scr20|199318|144938|114232   |110030|
|tai25b|868229041|465102132|345475531    |344355646|


2. mit Berücksichtigung von Mindestabstände


|Datei|FCFS|ROS|ILS|
|---|---|---|----|
|scr12|-|35662|34800|
|scr15|-|69172|63128|
|scr20|-|177008|120878|
|tai25b|868229041|-|507556428|

In der zweiten Tabelle bedeutet die Stelle, an der der Strich gezeichnet ist, dass ich durch den entsprechenden Algorithmus(Spalte Name) keine Lösung erhalten kann. Die initiale Permutationen von scr12, scr15 und scr20 würden alle zu einem Abstand zwischen Anlagen führen, der kleiner als der kürzeste Abstand(M Matrix) ist, daher ist dies nicht zulässig. Aber alle drei Situationen können die Startlösung über ROS erhalten.

Die Datei tai25b hingegen stellt das Gegenteil dar. Ihre Initiallösung ist zulässig, aber aufgrund der großen Anzahl von Anlagen kann ich den heuristischen Algorithmus nicht verwenden, um die Startlösung in kurzer Zeit zu erhalten. Also versuche ich, die folgenden zwei Möglichkeiten zu verwenden:

1. Nehmen das FCFS-Lösung als Eingabe des ILS-Algorithmus und optimieren sie.
2. In Ermangelung einer zulässigen Startlösung direkt eine nicht zulässige Lösung in den ILS-Algorithmus eingeben


In [110]:
# tai25b: InitialPermutation ist zulässig mit Berücksichtigung von Mindestabstand
tai25b_Start = Solution(output_25_m, output_25_m.InitialPermutation)
EvaluationLogic(True).DefineTotalTransport(tai25b_Start)

print(tai25b_Start)
print(tai25b_Start.Available)

tai25b_Start.CheckAvailable()
print(tai25b_Start.Available)

Die Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24] führt zu einer Transportleistung von 868229041
True
True


Die Ausgabe des obigen Codeblocks bestätigt, dass die initiale Lösung ( fcfs) für die Datei tai25b die einzige zulässige der vier Dateien ist.

In [111]:
# vier initiale Lösungen ohne Berücksichtigung von Mindestabstand
print(output_12.CalculateInitialTransport())
print(output_15.CalculateInitialTransport())
print(output_20.CalculateInitialTransport())
print(output_25.CalculateInitialTransport())

50116
77278
199318
868229041


In [112]:
# Initiale Lösungen von scr_15, scr_20 und tai_25b, ohne Berücksichtigung von M
# Diese Ergebnisse sind noch weit von einer wirklich optimalen Lösung entfernt.

solver = Solver(data, 999, False)
ROS_12 = solver.ConstructionPhase()

solver_15 = Solver(data_15, 999, False)
ROS_15 = solver_15.ConstructionPhase()

solver_20 = Solver(data_20, 999, False)
ROS_20 = solver_20.ConstructionPhase()

solver_25 = Solver(data_25, 999, False)
ROS_25 = solver_25.ConstructionPhase()

Generating an initial solution according to ROS. 
Constructive Solution found. 
Generating an initial solution according to ROS. 
Constructive Solution found. 
Generating an initial solution according to ROS. 
Constructive Solution found. 
Generating an initial solution according to ROS. 
Constructive Solution found. 


In [113]:
print(ROS_12)
print(ROS_15)
print(ROS_20)
print(ROS_25)

Die Permutation [ 9 11  4 10  2  6  7  3  8  1  0  5] führt zu einer Transportleistung von 34242
Die Permutation [ 1  9 10  8 14  0 12 13  2  5  3  4 11  6  7] führt zu einer Transportleistung von 60208
Die Permutation [13  7 18 11  1 15  6  2 17  8 14 16  5  4  0  9 12 19 10  3] führt zu einer Transportleistung von 144938
Die Permutation [12  6  9 22  5  2  8 18 17  3 21  4 16 19 14  0 10 23 11 15  7 13 24  1
 20] führt zu einer Transportleistung von 465102132


In [114]:
# Initiale Lösungen von scr_15, scr_20 und tai_25b, mit Berücksichtigung von M   

solver_m = Solver(data, 999, True)
ROS_12_m = solver_m.ConstructionPhase()

solver_15 = Solver(data_15, 999, True)
ROS_15_m = solver_15.ConstructionPhase()

solver_20 = Solver(data_20, 999, True)
ROS_20_m = solver_20.ConstructionPhase()

# tai25b hat keine optimale Lösung für ROS, wenn man den Mindestabstand berücksichtigt, daher wird keine Ausgabe benötigt


Generating an initial solution according to ROS. 
Constructive Solution found. 
Generating an initial solution according to ROS. 
Constructive Solution found. 
Generating an initial solution according to ROS. 
Constructive Solution found. 


In [115]:
print(ROS_12_m)
print(ROS_15_m)
print(ROS_20_m)

Die Permutation [ 8  6  0  7 10  1  5  9  4  2 11  3] führt zu einer Transportleistung von 35662
Die Permutation [10  2 14  6  7  4  5  1 11  9  0  8 12 13  3] führt zu einer Transportleistung von 69172
Die Permutation [ 0 13  4  5 18  9 10 14 11 12 15  2  1 16  6  7  3 17  8 19] führt zu einer Transportleistung von 177008


In [116]:
# Durch ROS kann ich nur eine unzulässige Lösung bekommen
solver_25 = Solver(data_25, 999, True)
print(solver_25.ConstructionPhase())

Generating an initial solution according to ROS. 
Constructive Solution found. 
Die Permutation [6, 3, 10, 8, 15, 11, 18, 12, 24, 21, 5, 2, 4, 14, 20, 9, 17, 23, 13, 16, 0, 19, 1, 7, 22] führt zu einer Transportleistung von 999999999


# class BaseNeighborhood

Nun beginnt das Näherungsverfahren für die initiale Lösung. Der erste Schritt besteht darin, eine BaseNeighborhood-klasse zu erstellen, in der wir versuchen, auf der Grundlage der ursprünglichen Lösung eine lokale Suche innerhalb ihrer Nachbarschaft durchzuführen. Ich habe SwapMove als Methode gewählt, um die Nachbarschaft der aktuellen Lösung zu erstellen und das SwapMove-Objekt in der Klasse SwapNeighborhood zu konstruieren.

In der Klasse BaseNeighborhood benötigen wir zunächst eine DiscoverMoves-Funktion, um alle Nachbarschaftslösungen zu ermitteln. Diese Funktion muss jedoch aufgrund der abstrakten Klasse in der Unterklasse SwapNeighborhood aufgerufen werden.

Die Funktion EvaluateMove verwendet lediglich die EvaluationLogic-Klasse, um die Gesamttransportleistungen für die aktuelle moveSolution zu berechnen und gibt diese moveSolution zurück. Sie wird in der Funktion EvaluateMovesBestImprovement aufgerufen und muss nicht separat aufgerufen werden.

Die Funktion EvaluateMovesBestImprovement durchläuft einfach die Liste der moves, die von der Funktion DiscoverMoves erzeugt wurde, und erstellt daraus ein Solution-objekt, das dann der Liste MoveSolutions hinzugefügt wird.
Wir sortieren die moveSolutions-Liste in aufsteigender Reihenfolge nach der Gesamtzahl der Transporte in der MakeBestMove-Funktion und geben dann das 0te Element der Liste zurück. D.h. finden Sie die Lösung mit der geringsten Gesamttransportleistung unter allen MoveSolutions

Die update() Funktion wird verwendet, um das Attribut Permutation zu ändern und die moves und MoveSolutions auf der Grundlage der vorherigen Permutation zu löschen

LocalSearch ist eine einzelne lokale Suche in der entsprechenden Nachbarschaft, um die lokal optimale Lösung in dieser Nachbarschaft zu finden
IteratedLocalSearch ist eine lokale Suche, die mehrere Iterationen durch Hinzufügen von Störungen durchläuft, bis die Stop- oder Akzeptanzbedingungen der Iteration erreicht sind


### class SwapMove and class SwapMoveNeghborhood

Die Klasse SwapMove definiert einen solchen Generalisierungsprozess, indem sie zwei beliebige Elemente(Indexe indexA bzw. indexB) einer Permutation vertauscht.

Die SwapMove-Objekte werden dann mit der Funktion DiscoverMoves in der Klasse SwapNeighborhood konstruiert, und diese Objekte werden dem Attribut Moves hinzugefügt, das SwapNeighborhood von seiner Superklasse BaseNeighborhood erbt.

Außerdem habe ich mir eine andere Methode ausgedacht, um die Nachbarschaft zu erhalten: TwoEdgeExchange. Aber nach meinen Experimenten habe ich gelernt, dass die TwoEdgeExchange-Methode für die vier vorliegenden Instanzen nicht geeignet ist. Mit anderen Worten: In den mit dem TwoEdgeExchange erstellten Nachbarschaften waren die erzielten Lösungen immer schlechter als beim Swopmove.

Daher habe ich in meinem Programm den Aufruf der TwoEdgeExchange-Methode entfernt. Aber ich kann die Gültigkeit der TwoEdgeExchange-Methode für andere Fälle nicht bestreiten.


In [117]:
class BaseNeighborhood():
    def __init__(self, inputData, initialPermutation, evaluationLogic, solutionPool):
        self.InputData = inputData
        self.OutputData = OutputData(inputData)
        self.Permutation = initialPermutation
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool

        self.Moves = []
        self.MoveSolutions = []

        self.Type = 'None'

    # DiscoverMoves wird in Unterklasse SwapNeighborhood aufgerufen
    def DiscoverMoves(self):
        raise Exception('DiscoverMoves is implemented in the two child classes. ')

    # Diese Funktion berechnet einfach die GesamtTransporte für die aktuelle moveSolution unter Verwendung der EvaluationLogic-Klasse 
    # und gibt diese moveSolution zurück. 
    # Die Funktion wird in der Funktion EvaluateMovesBestImprovement aufgerufen und nicht separat aufgerufen werden muss.
    def EvaluateMove(self, move): 
        moveSolution = Solution(self.OutputData, move.Permutation)
        self.EvaluationLogic.DefineTotalTransport(moveSolution)
        return moveSolution

    # Die Funktion EvaluateMovesBestImprovement durchläuft einfach die Liste der moves, die von der Funktion DiscoverMoves generiert wurden. 
    # Dann werden moves als Solution konstruiert und dieses dann der Liste MoveSolutions hinzugefügt .
    def EvaluateMovesBestImprovement(self):
        for move in self.Moves:
            moveSolution = self.EvaluateMove(move)
            self.MoveSolutions.append(moveSolution)

    def MakeBestMove(self):
        self.MoveSolutions.sort(key = lambda solution: solution.TotalTransport) 
        bestNeighborhoodSolution = self.MoveSolutions[0]
        return bestNeighborhoodSolution


    def Update(self, permutation): 
        self.Permutation = permutation

        self.Moves.clear()
        self.MoveSolutions.clear()

    def LocalSearch(self, solution):
        hasSolutionImproved = True

        self.Update(solution.Permutation)
        self.DiscoverMoves()
        self.EvaluateMovesBestImprovement()

        
        bestNeighborhoodSolution = self.MakeBestMove()
        if bestNeighborhoodSolution.TotalTransport < solution.TotalTransport:
            print('New local optimal Solution has been found! ')
            print(bestNeighborhoodSolution)

            self.SolutionPool.AddSolution(bestNeighborhoodSolution)
            solution.Permutation = bestNeighborhoodSolution.Permutation
            solution.TotalTransport = bestNeighborhoodSolution.TotalTransport # 这里的赋值是为了进入下一轮while循环，局部搜索bestNeighborhoodSol的邻域

        else:
            print(f'Reached local optimum of {self.Type} neighborhood. Stop local search. ')
            hasSolutionImproved = False


class SwapMove:
    def __init__(self, initialPermutation, indexA, indexB):
        self.Permutation = list(initialPermutation)
        self.IndexA = indexA
        self.IndexB = indexB

        self.Permutation[indexA] = initialPermutation[indexB]
        self.Permutation[indexB] = initialPermutation[indexA]


class SwapNeighborhood(BaseNeighborhood):
    def __init__(self, inputData, initialPermutation, evaluationLogic, solutionPool):
        super().__init__(inputData, initialPermutation, evaluationLogic, solutionPool)

        self.Type = 'Swap'

    def DiscoverMoves(self):
        for i in range(len(self.Permutation)):
            for j in range(len(self.Permutation)):
                if i < j:
                    swapMove = SwapMove(self.Permutation, i, j)
                    self.Moves.append(swapMove)


class TwoEdgeExchangeMove:
    def __init__(self, initialPermutation, indexA, indexB):
        self.Permutation = []

        self.Permutation.extend(initialPermutation[: indexA])
        self.Permutation.extend(reversed(initialPermutation[indexA: indexB]))
        self.Permutation.extend(initialPermutation[indexB: ])

class TwoEdgeExchangeNeighborhood(BaseNeighborhood):
    def __init__(self, inputData, initialPermutation, evaluationLogic, solutionPool):
        super().__init__(inputData, initialPermutation, evaluationLogic, solutionPool)

        self.Type = 'TwoEdgeExchange'

    def DiscoverMoves(self):
        for i in range(len(self.Permutation)):
            for j in range(len(self.Permutation)):
                if j < i + 1:
                    continue
                twoEdgeMove = TwoEdgeExchangeMove(self.Permutation, i, j)
                self.Moves.append(twoEdgeMove)
        



### class ImprovementAlgorithm

Das Wichtigste in dieser Klasse ist die Erzeugung von Nachbarn für eine Lösung über die Funktion InitializeNeighborhoods. Es ist natürlich auch möglich, mehrere Nachbarschaften zu erstellen und diese in einem Wörterbuch als Schlüssel-Wert-Paare zu speichern. Ich verwende aber nur SwapMove, um die Nachbarschaften zu erstellen, was für die Lösung bereits optimal ist.



### class IterativeImprovement(ImprovementAlgorithm)

Die Klasse IterativeImprovement ist eine Unterklasse der Klasse ImprovementAlgorithm und verwendet die Funktion Run, um eine lokale Suche über alle Typen von Nachbarschaften durchzuführen und die optimale Lösung der Nachbarschaft zurückzugeben. Es ist wichtig zu beachten, dass dies nur eine einzige lokale Suche ist. Um die potentiele optimale Lösung zu finden, sind mehrere Iterationen der lokalen Suche erforderlich.

Nämlich die intensive Suche nach sehr guten Lösungen in vielversprechenden Bereichen der Lösungsraums.



# class IteratedLocalSearch(ImprovementAlgorithm)

IteratedLocalSearch ist im Wesentlichen durch eine Pertubationsfunktion, die der Permutation der Lösung einen Perturbationsterm hinzufügt, um eine lokale Suche in einem noch nicht lokal durchgesuchten Bereich des aktuellen Lösungsraums zu ermöglichen.

In der Funktion Pertubation wähle ich nach dem Zufallsprinzip zwei Positionen aus der aktuellen Permutation der Lösung aus, vertausche diese beiden Positionen und wiederhole die obigen Schritte, bis die maximale Anzahl der Iterationen erreicht ist oder die Akzeptanzbedingung erfüllt ist.

- Schritte：
1. Pertubation
2. in einen anderen Bereich des Lösungsraums springen
3. neue Lösung erzeugen
4. diese neue Lösung gilt als Startpunkt von local search 


Ich habe auch versucht, eine neue Funktion, Pertubation_3, zu definieren, um die Positionen der drei Elemente in der Permutation zu vertauschen. Nach meinen Experimenten führte dies nicht zu einer besseren Lösung, und der einzige Vorteil von Pertubation_3 war, dass es die Laufzeit des Programms verkürzte.

Die von der Funktion Pertubation_3 erzeugten Ergebnisse werden im Ordner Solutions_Pertubation3 gespeichert. Ich habe die Ausführung der Funktion Pertubation_3 aus meinem Code entfernt, da sie nicht zur Optimierung beiträgt.


In [118]:
class ImprovementAlgorithm:
    def __init__(self, inputData, neighborhoodTypes = ['Swap']):
        self.InputData = inputData

        self.EvaluationLogic = {}
        self.SolutionPool = {}
        self.RNG = {}

        self.NeighborhoodTypes = neighborhoodTypes
        self.Neighborhoods = {}

    def Initialize(self, evaluationLogic, solutionPool, rng = None):
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool
        self.RNG = rng


    def CreateNeighborhood(self, neighborhoodType, bestCurrentSolution):
        if neighborhoodType == 'Swap':
            return SwapNeighborhood(self.InputData, bestCurrentSolution.Permutation, self.EvaluationLogic, self.SolutionPool)
        elif neighborhoodType == 'TwoEdgeExchange':
            return TwoEdgeExchangeNeighborhood(self.InputData, bestCurrentSolution.Permutation, self.EvaluationLogic, self.SolutionPool)

        else:
            raise Exception(f'Neighborhood type {neighborhoodType} not defined. ')

    # Erstellen von Nachbarschaft, deren Typ ist Dictionary
    def InitializeNeighborhoods(self, solution):
        for neighborhoodType in self.NeighborhoodTypes: 
            neighborhood = self.CreateNeighborhood(neighborhoodType, solution)
            self.Neighborhoods[neighborhoodType] = neighborhood



class IterativeImprovement(ImprovementAlgorithm):
    def __init__(self, inputData, neighborhoodTypes = ['Swap']):
        super().__init__(inputData, neighborhoodTypes)

    def Run(self, solution):
        self.InitializeNeighborhoods(solution)  

        for neighborhoodType in self.NeighborhoodTypes:
            neighborhood = self.Neighborhoods[neighborhoodType]

            neighborhood.LocalSearch(solution)

        return solution


class IteratedLocalSearch(ImprovementAlgorithm):
    def __init__(self, inputData, maxIterations, neighborhoodTypes = ['Swap'], localSearchAlgorithm = None):
        super().__init__(inputData, neighborhoodTypes)

        self.MaxIterations = maxIterations
        self.NumberStationsToSwap = 2
        self.LocalSearchAlgorithm = localSearchAlgorithm

    def Initialize(self, evaluationLogic, solutionPool, rng): 
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool
        self.RNG = rng

        self.LocalSearchAlgorithm.Initialize(self.EvaluationLogic, self.SolutionPool)

    def Pertubation(self, currentSolution):   
        i, j = self.RNG.choice(list(range(self.InputData.Size)), size = self.NumberStationsToSwap, replace = False)
        swapPermutation = copy.deepcopy(currentSolution.Permutation)
        swapPermutation[i] = currentSolution.Permutation[j]
        swapPermutation[j] = currentSolution.Permutation[i]
        currentSolution.setPermutation(swapPermutation)
        self.EvaluationLogic.DefineTotalTransport(currentSolution)
        return currentSolution

    def Pertubation_3(self, currentSolution):   
        x, y, z = self.RNG.choice(list(range(self.InputData.Size)), size = 3, replace = False) 
        swapPermutation = copy.deepcopy(currentSolution.Permutation)

        swapPermutation[x] = currentSolution.Permutation[y]
        swapPermutation[y] = currentSolution.Permutation[z]
        swapPermutation[z] = currentSolution.Permutation[x]
        currentSolution.setPermutation(swapPermutation)
        self.EvaluationLogic.DefineTotalTransport(currentSolution)

        return currentSolution


    def Run(self, currentSolution):
        currentSolution = self.LocalSearchAlgorithm.Run(currentSolution)  
        currentBest = self.SolutionPool.GetLowestTransportSol().TotalTransport

        iteration = 0

        while (iteration < self.MaxIterations):
            newSolution = self.Pertubation(currentSolution) 
            newSolution = self.LocalSearchAlgorithm.Run(newSolution)
            newSolution.CheckAvailable() 
            if newSolution.TotalTransport < currentSolution.TotalTransport and newSolution.Available == True:
                currentSolution = newSolution

                if newSolution.TotalTransport < currentBest:

                    print(f'New best solution in iteration {iteration}: {currentSolution}')

                    self.SolutionPool.AddSolution(currentSolution)
                    currentBest = newSolution.TotalTransport

            iteration += 1

        return self.SolutionPool.GetLowestTransportSol()


Dieser unterstehende Codeblock dient dazu, die erste Nachbarschaft der zuvor über ROS generierten bestInitialSolution-Lösung zu erstellen und die potenziell optimale Lösung in dieser Nachbarschaft zu finden. Ich habe die Logik von ILS in Schritten dargestellt, der eigentliche Prozess, der hier gezeigt wird, ist der der Intensivierung. Das vollständige ILS erfordert auch eine Pertubation.

Der Vergleich zeigt, dass der Gesamttransport von bs nach der ersten Nachbarschaftssuche verbessert wird:

- Vor der LocalSearch:
- Die Permutation [10  5  6  9  1  8  4  0  2  7 11  3] führt zu einer Transportleistung von 33560

In [119]:
swapNeighbor = SwapNeighborhood(data, bestInitialSolution.Permutation, EvaluationLogic(False), SolutionPool())
swapNeighbor.LocalSearch(bestInitialSolution) 

New local optimal Solution has been found! 
Die Permutation [2, 5, 6, 9, 1, 8, 4, 0, 10, 7, 11, 3] führt zu einer Transportleistung von 33072


### Optimale Lösungen generieren durch SwapMove und TwoEdgeExchange

Als Nächstes beginne ich, die optimale Lösungen mithilfe meines Programms zu generieren. Mein Plan sieht folgendermaßen aus: Jede Instanz wird in solche unterteilt, die den Mindestabstand berücksichtigen, und solche, die ihn nicht berücksichtigen, und in jedem Fall werden zwei Methoden der Nachbarschaftssuche, SwapMove bzw. TwoEdgeExchange, verwendet.

Beim Vergleich der Ergebnisse habe ich festgestellt, dass die SwapMove-Methode in allen vier Instanzen bessere Ergebnisse liefert. Deshalb habe ich beim Exportieren meiner Berechnungen in json-Dateien nur die Ergebnisse der SwapMove-Methode ausgewählt und die Codeblocks von TwoEdgeExchange weggelassen.

Außerdem habe ich in der Instanz tai25b_mMA die FCFS-Ergebnisse als zu optimierende Lösung verwendet, da ich keine zulässige Lösung durch ILS finden konnte und schließlich eine optimierte Lösung gefunden habe.


Obwohl die Permutation, die ich in dem nachstehenden Codeblock generiere, von der wahren optimalen Lösung abweichen kann, kann man davon ausgehen, dass es mehrere optimale Lösungen gibt. Dies spiegelt sich auch im Ablauf meines Codes wider, d.h. dass es mehrere Lösungen gibt, die das minimale Gesamttransportvolumen erreichen.

In [120]:
# scr_12, ohne Mindestabstand, Swap
solver_12 = Solver(data, 2, False)
localSearch_12 = IterativeImprovement(data)
ils_12 = IteratedLocalSearch(data, 100, ['Swap'], localSearch_12)


outputFormat_12_swap = solver_12.RunLocalSearch(ils_12)

print('\n\n')
print(outputFormat_12_swap)


Generating an initial solution according to ROS. 
Constructive Solution found. 
New local optimal Solution has been found! 
Die Permutation [9, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 4] führt zu einer Transportleistung von 31410
New local optimal Solution has been found! 
Die Permutation [9, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 4] führt zu einer Transportleistung von 31410
New local optimal Solution has been found! 
Die Permutation [9, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 4] führt zu einer Transportleistung von 31410
New local optimal Solution has been found! 
Die Permutation [9, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 4] führt zu einer Transportleistung von 31410
New local optimal Solution has been found! 
Die Permutation [9, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 4] führt zu einer Transportleistung von 31410
New local optimal Solution has been found! 
Die Permutation [9, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 4] führt zu einer Transportleistung von 31410
New local optimal Solution has been found! 
Die Permutation [9, 6, 1

In [121]:
# meine Lösung prüfen
check12 = Solution(output_12, outputFormat_12_swap.Permutation)
EvaluationLogic().DefineTotalTransport(check12)
print(check12)


Die Permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] führt zu einer Transportleistung von 31410


In [122]:
# scr_12, mit Mindestabstand, Swap

solver_12_m= Solver(data, 2, True)  #2
localSearch_12_m = IterativeImprovement(data)
ils_12_m = IteratedLocalSearch(data, 200, ['Swap'], localSearch_12_m)

# outputFormat_12_m_swap hier steht "_m" für die Berücksichtigung von M-Matrix
outputFormat_12_m_swap = solver_12_m.RunLocalSearch(ils_12_m)

print('\n\n')
print(outputFormat_12_m_swap)


Generating an initial solution according to ROS. 
Constructive Solution found. 
New local optimal Solution has been found! 
Die Permutation [3, 5, 1, 9, 4, 10, 6, 2, 0, 7, 8, 11] führt zu einer Transportleistung von 35946
New local optimal Solution has been found! 
Die Permutation [3, 5, 1, 9, 4, 10, 6, 2, 0, 7, 8, 11] führt zu einer Transportleistung von 35946
New local optimal Solution has been found! 
Die Permutation [3, 5, 1, 9, 4, 10, 6, 2, 0, 7, 8, 11] führt zu einer Transportleistung von 35946
New local optimal Solution has been found! 
Die Permutation [3, 5, 1, 9, 4, 10, 6, 2, 0, 7, 8, 11] führt zu einer Transportleistung von 35946
New local optimal Solution has been found! 
Die Permutation [3, 5, 1, 9, 4, 10, 6, 2, 0, 7, 8, 11] führt zu einer Transportleistung von 35946
New local optimal Solution has been found! 
Die Permutation [3, 5, 1, 9, 4, 10, 6, 2, 0, 7, 8, 11] führt zu einer Transportleistung von 35946
New local optimal Solution has been found! 
Die Permutation [3, 5, 1

In [123]:
# meine Lösung sowie die Zulässigkeit überprüfen
check12_m = Solution(output_12, outputFormat_12_m_swap.Permutation)
print(EvaluationLogic(True).DefineTotalTransport(check12_m))
check12_m.CheckAvailable()
print(check12_m.Available)
print(check12_m.Permutation)

34556
True
[3, 5, 1, 4, 10, 2, 6, 7, 0, 9, 11, 8]


In [124]:
# scr_15, ohne Mindestabstand, Swap

solver_15 = Solver(data_15, 2022, False)
LS_15 = IterativeImprovement(data_15)
ILS_15 = IteratedLocalSearch(data_15, 250, ['Swap'], LS_15)

outputFormat_15_swap = solver_15.RunLocalSearch(ILS_15)

print('\n\n')
print(outputFormat_15_swap)


Generating an initial solution according to ROS. 
Constructive Solution found. 
New local optimal Solution has been found! 
Die Permutation [14, 8, 9, 12, 1, 5, 4, 0, 13, 11, 2, 7, 6, 10, 3] führt zu einer Transportleistung von 59104
New local optimal Solution has been found! 
Die Permutation [14, 8, 9, 12, 1, 5, 4, 0, 13, 11, 2, 7, 6, 10, 3] führt zu einer Transportleistung von 59104
New local optimal Solution has been found! 
Die Permutation [14, 8, 9, 12, 1, 5, 4, 0, 13, 11, 2, 7, 6, 10, 3] führt zu einer Transportleistung von 59104
New local optimal Solution has been found! 
Die Permutation [14, 8, 9, 12, 1, 5, 4, 0, 13, 11, 2, 7, 6, 10, 3] führt zu einer Transportleistung von 59104
New local optimal Solution has been found! 
Die Permutation [14, 8, 9, 12, 1, 5, 4, 0, 13, 11, 2, 7, 6, 10, 3] führt zu einer Transportleistung von 59104
New local optimal Solution has been found! 
Die Permutation [14, 8, 9, 12, 1, 5, 4, 0, 13, 11, 2, 7, 6, 10, 3] führt zu einer Transportleistung von 59

In [125]:
# meine Lösung prüfen
check15 = Solution(output_15, outputFormat_15_swap.Permutation)
EvaluationLogic().DefineTotalTransport(check15)
print(check15)

Die Permutation [7, 5, 9, 1, 12, 0, 4, 8, 13, 6, 3, 2, 11, 10, 14] führt zu einer Transportleistung von 53114


In [126]:
# scr_15, mit Mindestabstand, Swap

solver_15_m = Solver(data_15, 2, True)
LS_15_m = IterativeImprovement(data_15)
ILS_15_m = IteratedLocalSearch(data_15, 250, ['Swap'], LS_15_m)

outputFormat_15_m_swap = solver_15_m.RunLocalSearch(ILS_15_m)

print('\n\n')
print(outputFormat_15_m_swap)

Generating an initial solution according to ROS. 
Constructive Solution found. 
New local optimal Solution has been found! 
Die Permutation [3, 5, 2, 4, 8, 14, 9, 13, 1, 10, 0, 11, 7, 6, 12] führt zu einer Transportleistung von 65718
New local optimal Solution has been found! 
Die Permutation [3, 8, 2, 4, 14, 5, 9, 13, 1, 10, 0, 11, 7, 6, 12] führt zu einer Transportleistung von 63242
New local optimal Solution has been found! 
Die Permutation [0, 8, 1, 4, 14, 5, 9, 13, 2, 10, 3, 11, 7, 6, 12] führt zu einer Transportleistung von 63128
New local optimal Solution has been found! 
Die Permutation [0, 8, 1, 4, 14, 5, 9, 13, 2, 10, 3, 11, 7, 6, 12] führt zu einer Transportleistung von 63128
New local optimal Solution has been found! 
Die Permutation [0, 8, 1, 4, 14, 5, 9, 13, 2, 10, 3, 11, 7, 6, 12] führt zu einer Transportleistung von 63128
New local optimal Solution has been found! 
Die Permutation [0, 8, 1, 4, 14, 5, 9, 13, 2, 10, 3, 11, 7, 6, 12] führt zu einer Transportleistung von 63

In [127]:
# meine Lösung sowie die Zulässigkeit überprüfen
check15_m = Solution(output_15_m, outputFormat_15_m_swap.Permutation)
print(EvaluationLogic(True).DefineTotalTransport(check15_m))
check15_m.CheckAvailable()
print(check15_m.Available)
print(check15_m.Permutation)

63128
True
[0, 8, 1, 4, 14, 5, 9, 13, 2, 10, 3, 11, 7, 6, 12]


In [128]:
# scr_20, ohne Mindestabstand, Swap
solver_20 = Solver(data_20, 222, False)
LS_20 = IterativeImprovement(data_20)
ILS_20 = IteratedLocalSearch(data_20, 250, ['Swap'], LS_20)

outputFormat_20_swap = solver_20.RunLocalSearch(ILS_20)

print('\n\n')
print(outputFormat_20_swap)

Generating an initial solution according to ROS. 
Constructive Solution found. 
New local optimal Solution has been found! 
Die Permutation [12, 17, 8, 18, 19, 16, 13, 10, 14, 0, 5, 3, 7, 4, 1, 6, 2, 15, 11, 9] führt zu einer Transportleistung von 143772
New local optimal Solution has been found! 
Die Permutation [12, 17, 8, 18, 19, 16, 13, 10, 5, 0, 11, 3, 7, 4, 1, 6, 2, 15, 14, 9] führt zu einer Transportleistung von 139952
New local optimal Solution has been found! 
Die Permutation [12, 17, 8, 18, 19, 16, 13, 10, 5, 0, 11, 3, 7, 4, 1, 6, 2, 15, 14, 9] führt zu einer Transportleistung von 139952
New local optimal Solution has been found! 
Die Permutation [12, 17, 8, 18, 19, 16, 13, 10, 5, 0, 11, 3, 7, 4, 1, 6, 2, 15, 14, 9] führt zu einer Transportleistung von 139952
New local optimal Solution has been found! 
Die Permutation [12, 17, 8, 18, 19, 16, 13, 10, 5, 0, 11, 3, 7, 4, 1, 6, 2, 15, 14, 9] führt zu einer Transportleistung von 139952
New local optimal Solution has been found! 
D

In [129]:
# meine Lösung prüfen
check20 = Solution(output_20, outputFormat_20_swap.Permutation)
EvaluationLogic().DefineTotalTransport(check20)
print(check20)

Die Permutation [7, 18, 12, 17, 10, 16, 19, 15, 0, 8, 3, 9, 6, 4, 5, 2, 1, 14, 13, 11] führt zu einer Transportleistung von 114232


In [130]:
# scr_20, mit Mindestabstand, Swap
solver_20_m = Solver(data_20, 222, True)
LS_20_m = IterativeImprovement(data_20)
ILS_20_m = IteratedLocalSearch(data_20, 250, ['Swap'], LS_20_m)

outputFormat_20_m_swap = solver_20_m.RunLocalSearch(ILS_20_m)

print('\n\n')
print(outputFormat_20_m_swap)

Generating an initial solution according to ROS. 
Constructive Solution found. 
New local optimal Solution has been found! 
Die Permutation [3, 12, 11, 13, 18, 6, 5, 4, 2, 19, 1, 14, 17, 10, 0, 9, 7, 16, 15, 8] führt zu einer Transportleistung von 169352
New local optimal Solution has been found! 
Die Permutation [3, 12, 11, 13, 18, 6, 5, 4, 2, 19, 1, 14, 17, 10, 0, 9, 7, 16, 15, 8] führt zu einer Transportleistung von 169352
New local optimal Solution has been found! 
Die Permutation [3, 12, 11, 13, 15, 6, 5, 4, 2, 19, 1, 14, 17, 7, 0, 9, 10, 16, 18, 8] führt zu einer Transportleistung von 165232
New local optimal Solution has been found! 
Die Permutation [3, 12, 11, 13, 15, 6, 5, 4, 2, 19, 1, 14, 17, 7, 0, 9, 10, 16, 18, 8] führt zu einer Transportleistung von 165232
New local optimal Solution has been found! 
Die Permutation [3, 12, 11, 13, 15, 6, 5, 4, 2, 19, 1, 18, 10, 7, 0, 9, 17, 16, 14, 8] führt zu einer Transportleistung von 158824
New local optimal Solution has been found! 
D

In [131]:
# meine Lösung sowie die Zulässigkeit überprüfen
check20_m = Solution(output_20_m, outputFormat_20_m_swap.Permutation)
print(EvaluationLogic(True).DefineTotalTransport(check20_m))
check20_m.CheckAvailable()
print(check20_m.Available)
print(check20_m.Permutation)

120878
True
[2, 17, 15, 18, 4, 19, 13, 12, 11, 6, 0, 10, 3, 7, 8, 1, 9, 5, 14, 16]


In [132]:
# tai25b, ohne Mindestabstand, Swap
solver_25 = Solver(data_25, 7, False)
LS_25 = IterativeImprovement(data_25)
ILS_25 = IteratedLocalSearch(data_25, 1000, ['Swap'], LS_25)

outputFormat_25_swap = solver_25.RunLocalSearch(ILS_25)


print('\n\n')
print(outputFormat_25_swap)
# 设随机种子为2022，tai25b可得最优解
# [3, 14, 9, 8, 12, 4, 24, 18, 6, 2, 16, 5, 17, 19, 15, 1, 21, 22, 7, 10, 20, 23, 13, 11, 0] führt zu einer Transportleistung von 344355646

Generating an initial solution according to ROS. 
Constructive Solution found. 
New local optimal Solution has been found! 
Die Permutation [23, 12, 17, 14, 9, 2, 6, 8, 15, 3, 24, 4, 5, 16, 18, 1, 7, 10, 20, 13, 21, 19, 22, 0, 11] führt zu einer Transportleistung von 395138716
New local optimal Solution has been found! 
Die Permutation [19, 12, 9, 14, 17, 2, 6, 8, 15, 3, 24, 4, 5, 16, 18, 1, 7, 10, 20, 13, 21, 23, 22, 0, 11] führt zu einer Transportleistung von 385397444
New local optimal Solution has been found! 
Die Permutation [19, 12, 9, 14, 17, 2, 6, 8, 15, 3, 24, 4, 5, 16, 18, 1, 7, 10, 20, 13, 21, 23, 22, 0, 11] führt zu einer Transportleistung von 385397444
New local optimal Solution has been found! 
Die Permutation [19, 12, 9, 14, 17, 2, 6, 8, 15, 3, 24, 4, 5, 16, 18, 1, 7, 10, 20, 13, 21, 23, 22, 0, 11] führt zu einer Transportleistung von 385397444
New local optimal Solution has been found! 
Die Permutation [19, 3, 9, 14, 18, 2, 6, 8, 15, 12, 24, 4, 5, 16, 17, 1, 7, 10, 20, 

In [133]:
# meine Lösung prüfen
check25 = Solution(output_25, outputFormat_25_swap.Permutation)
EvaluationLogic().DefineTotalTransport(check25)
print(check25)

Die Permutation [3, 9, 5, 12, 6, 17, 23, 14, 8, 16, 24, 15, 4, 19, 2, 1, 21, 22, 7, 11, 20, 10, 13, 0, 18] führt zu einer Transportleistung von 345475531


Ich war nicht in der Lage, mit dem ROS-Algorithmus eine zulässige Startlösung für die Instanz tai25b zu erzeugen.

Es gibt zwei Möglichkeiten, wie ich dieses Problem lösen kann.

Die erste besteht darin, die FCFS-Lösung als Eingabe für den ILS-Algorithmus zu verwenden, so dass durch Nachbarschaftssuche eine verbesserte Lösung gefunden werden kann.

Die zweite Möglichkeit besteht darin, eine nicht durchführbare Lösung als Eingabe für den ILS-Algorithmus zu verwenden. Durch die Verwendung verschiedener Zufallszahlen kann ILS auch zulässige Lösungen finden.

Da verschiedene Zufallszahlen unterschiedliche Auswirkungen auf verschiedene Instanzen haben, ist dieses Schema nicht universell. Es gibt Fälle, in denen eine Zufallszahl die optimale Lösungsausgabe für Instanz A garantiert, aber nicht für Instanz B gilt.

Ich habe dies nur getan, um die Vor- und Nachteile abzuwägen, sonst hätte ich den Optimierungsprozess nicht zu Ende führen können.



In [134]:
# tai25b, mit Mindestabstand, Swap
solver_25_m = Solver(data_25, 7, True)
LS_25_m = IterativeImprovement(data_25)
ILS_25_m = IteratedLocalSearch(data_25, 500, ['Swap'], LS_25_m)
# ILS_25_m.Initialize(solver_25_m.EvaluationLogic, solver_25_m.SolutionPool, solver_25_m.RNG)

outputFormat_25_m_swap = solver_25_m.RunLocalSearch(ILS_25_m)

print('\n\n')
print(outputFormat_25_m_swap)

Generating an initial solution according to ROS. 
Constructive Solution found. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of Swap neighborhood. Stop local search. 
Reached local optimum of

In [135]:
#  Möglichkeit 1. tai25b, initiale Permutation aus FCFS
tai25b_Start = Solution(output_25_m, output_25_m.InitialPermutation)
EvaluationLogic(True).DefineTotalTransport(tai25b_Start)
print(tai25b_Start)


solver_25_start = Solver(data_25, 2022, True)
LS_25_m = IterativeImprovement(data_25)
ILS_25_m_start = IteratedLocalSearch(data_25, 1000, ['Swap'], LS_25_m)

outputFormat_25_m = solver_25_start.ImprovementPhase(tai25b_Start, ILS_25_m_start)
print(outputFormat_25_m.Permutation)


Die Permutation [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24] führt zu einer Transportleistung von 868229041
New local optimal Solution has been found! 
Die Permutation [0, 1, 16, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 2, 17, 18, 19, 20, 21, 22, 23, 24] führt zu einer Transportleistung von 675806539
New local optimal Solution has been found! 
Die Permutation [0, 1, 16, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 2, 17, 18, 19, 20, 21, 22, 23, 24] führt zu einer Transportleistung von 675806539
New local optimal Solution has been found! 
Die Permutation [0, 1, 16, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 2, 17, 18, 19, 20, 21, 22, 23, 24] führt zu einer Transportleistung von 675806539
New local optimal Solution has been found! 
Die Permutation [0, 1, 16, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 2, 17, 18, 19, 20, 21, 22, 23, 24] führt zu einer Transportleistung von 675806539
New local optimal Solution has been found! 
Die Permut

In [136]:
sol_unzu = Solution(output_25_m, outputFormat_25_m_swap.Permutation)
print(sol_unzu)

Die Permutation [20, 4, 13, 8, 7, 17, 2, 15, 12, 10, 24, 18, 0, 16, 1, 6, 3, 11, 22, 23, 21, 19, 14, 9, 5] führt zu einer Transportleistung von -1


Wenn ich versuche, eine nicht zulässige Lösung als Eingabe für den ILS-Algorithmus zu verwenden, kann ILS auch keine zulässige Lösung erzeugen und der Compiler meldet einen Fehler. Der Grund dafür ist, dass der SolutionPool leer ist, so dass die Verwendung von [0] zu einem Indexfehler führt.

In [137]:
#  Möglichkeit 2. tai25b, unzulässige Solution als input von ILS
# tai25b, mit Mindestabstand, Swap

# unzulässige Lösung:


# tai25b_Start_unzu = Solution(output_25_m, outputFormat_25_m_swap.Permutation)
# EvaluationLogic(True).DefineTotalTransport(tai25b_Start_unzu)
# print(tai25b_Start_unzu)


# solver_25_start_unzu = Solver(data_25, 2022, True)
# LS_25_m_unzu = IterativeImprovement(data_25)
# ILS_25_m_start_unzu = IteratedLocalSearch(data_25, 1000, ['Swap'], LS_25_m_unzu)

# outputFormat_25_m_unzu = solver_25_start_unzu.ImprovementPhase(tai25b_Start_unzu, ILS_25_m_start_unzu)
# print(outputFormat_25_m_unzu.Permutation)

# print('\n\n')




In [138]:
# Die Zulässigkeit überprüfen

test_with_M = Solution(output_25_m, outputFormat_25_m.Permutation)
test_with_M.CheckAvailable()
print(test_with_M.ArrangedDMatrix)
print(test_with_M.MindDistanceMatrix)
print(test_with_M.Available)

[[    0     0     4     0     0   346     2     0    43 30468     0   581
      0 21429    41 11011     0     5    33     0     0    20   917 20865
      0]
 [    0     0  2544    14 37455     7  2137 54696     2   263   122  3873
      0     0     8     0  8144     0     0   218    69     0     0     0
      0]
 [    0     8     0 22120     0     0 44943   184    33     0     0     0
  34014     0     0     0     0    33     3     1  2951     0     0     0
      0]
 [    0     0   115     0  9687     0     0     0     0 15485    47   256
    116     0     2  1858     0     0    31 25326     9 25715     1     0
   3500]
 [    0 53531     0    87     0     0     0     3     0 47827     0 45654
      0  3839     1    41     0   623     0     0     0     0  2862     0
      2]
 [    0     0     0  2060     0     0    16  3904 12025 31971  3001     0
     13   389   539     0   635     0   125     0     4  5806     0     2
    374]
 [ 7437     0 28524  3362     0    92     0 27831     0   

In [139]:
# ROS kann keine Startsolution generieren
solver_25_m = Solver(data_25, 999, True)
solution_25_m = solver_25_m.ConstructionPhase()


Generating an initial solution according to ROS. 
Constructive Solution found. 


In [140]:
print(solution_25_m.Permutation)
print(solution_25_m.TotalTransport)

[6, 3, 10, 8, 15, 11, 18, 12, 24, 21, 5, 2, 4, 14, 20, 9, 17, 23, 13, 16, 0, 19, 1, 7, 22]
999999999


Ich habe das Ergebnis von FCFS als Startsolution von ILS genommen und nach der Ausführung des obigen Codeblocks festgestellt, dass das TotalTransport optimiert war.


|Datei|FCFS|ROS|ILS|
|---|---|---|----|
|scr12|-|35662|34556|
|scr15|-|69172|63128|
|scr20|-|177008|120878|
|tai25b|868229041|-|507556428|

- Für die Ausgabe werden insgesamt 8 json-Dateien, d.h. 4 Instanzen, benötigt, jeweils mit und ohne Berücksichtigung der M-Matrix

- Um in die json-Dateien zu schreiben, müssen zunächst das Dictionaries erstellen, das zwei key-value-Paare enthält. z.B. ObjVal：31140， Permutation： [1,2,3,4]
- Achtung: Permutation beginnt mit 1 statt 0

In [141]:
output_oma = [outputFormat_12_swap, outputFormat_15_swap, outputFormat_20_swap, outputFormat_25_swap]
output_mma = [outputFormat_12_m_swap, outputFormat_15_m_swap, outputFormat_20_m_swap, outputFormat_25_m]

In [142]:
print(outputFormat_25_m.Permutation)
print(outputFormat_25_m.TotalTransport)

[24, 17, 6, 3, 4, 5, 8, 2, 18, 9, 10, 16, 12, 13, 14, 15, 20, 1, 7, 19, 22, 21, 11, 23, 0]
507556428


# Ergebnisse

In [143]:
# Hier wird gezeigt, der Vergleich FCFS vs. ROS vs. ILS vs. OPT, ohne Mindestabstand
import pandas as pd
dict_oma = {"Datei": ['scr12', 'scr15', 'scr20', 'tai25b'], 
"FCFS": [output_12.CalculateInitialTransport(), output_15.CalculateInitialTransport(), output_20.CalculateInitialTransport(), output_25.CalculateInitialTransport()],
"ROS": [ROS_12.TotalTransport, ROS_15.TotalTransport, ROS_20.TotalTransport, ROS_25.TotalTransport], 
"ILS": [outputFormat_12_swap.TotalTransport, outputFormat_15_swap.TotalTransport, outputFormat_20_swap.TotalTransport, outputFormat_25_swap.TotalTransport],
"OPT":[31410, 51140, 110030, 344355646]}

df_oma = pd.DataFrame(dict_oma, index = dict_oma['Datei'])
print(df_oma)



         Datei       FCFS        ROS        ILS        OPT
scr12    scr12      50116      34242      31410      31410
scr15    scr15      77278      60208      53114      51140
scr20    scr20     199318     144938     114232     110030
tai25b  tai25b  868229041  465102132  345475531  344355646


Anhand der Vergleiche in der folgenden Tabelle, insbesondere der letzten Spalte, erhalte ich die optimale Lösung für den Fall scr_12, während das Ergebnis für tai25b nur um knapp 0,8 % von der optimalen Lösung abweicht. Allerdings weichen die Ergebnisse für scr_15 und scr_20 um 3.7 % bzw. 3.6 % von der optimalen Lösung ab, was ich für akzeptabel halte, auch wenn die Optimierung nicht so gut ist wie in den anderen Fällen.

In [144]:
df_oma['FCFS_to_ROS'] = (df_oma['FCFS'] - df_oma['ROS']) / df_oma['FCFS']
df_oma['ROS_to_ILS'] = (df_oma['ROS'] - df_oma['ILS']) / df_oma['ROS']
df_oma['ILS_to_OPT'] = (df_oma['ILS'] - df_oma['OPT']) / df_oma['ILS']

print(df_oma)

         Datei       FCFS        ROS        ILS        OPT  FCFS_to_ROS  \
scr12    scr12      50116      34242      31410      31410     0.316745   
scr15    scr15      77278      60208      53114      51140     0.220891   
scr20    scr20     199318     144938     114232     110030     0.272830   
tai25b  tai25b  868229041  465102132  345475531  344355646     0.464309   

        ROS_to_ILS  ILS_to_OPT  
scr12     0.082705    0.000000  
scr15     0.117825    0.037165  
scr20     0.211856    0.036785  
tai25b    0.257205    0.003242  


In [145]:
# Hier wird gezeigt, der Vergleich StartSolution(FCFS) vs. ILS, mit Mindestabstand
dict_mma = {"Datei": ['scr12', 'scr15', 'scr20', 'tai25b'], 
"StartSolution": [ROS_12_m.TotalTransport, ROS_15_m.TotalTransport, ROS_20_m.TotalTransport, output_25_m.CalculateInitialTransport()], 
"ILS": [outputFormat_12_m_swap.TotalTransport, outputFormat_15_m_swap.TotalTransport, outputFormat_20_m_swap.TotalTransport, outputFormat_25_m.TotalTransport],}

df_mma = pd.DataFrame(dict_mma)
print(df_mma)


    Datei  StartSolution        ILS
0   scr12          35662      34556
1   scr15          69172      63128
2   scr20         177008     120878
3  tai25b      868229041  507556428


Wie aus der nachstehenden Tabelle ersichtlich ist, liegt die mit dem ROS-Algorithmus ermittelte Startlösung bei kleinen Datenmengen (12 und 15) sehr nahe an der Lösung nach der iterativen Optimierung mit dem ILS-Algorithmus. Wenn die Datenmenge jedoch größer ist, wird der Optimierungsgrad des metaheuristischen Algorithmus erheblich verbessert

In [146]:
df_mma['Start_to_ILS'] = (df_mma['StartSolution'] - df_mma['ILS']) / df_mma['StartSolution']
print(df_mma)

    Datei  StartSolution        ILS  Start_to_ILS
0   scr12          35662      34556      0.031013
1   scr15          69172      63128      0.087376
2   scr20         177008     120878      0.317104
3  tai25b      868229041  507556428      0.415412


### Zusammenfassung
Insgesamt bin ich mit den optimierten Ergebnissen zufrieden.

Einerseits sind die Abweichungen meiner Ergebnisse von der wahren optimalen Lösung gering, wenn die Mindestabstandsmatrix nicht berücksichtigt wird: 0 %, 3 %, 3 % bzw. 0,3 %.

Andererseits habe ich bei der Betrachtung der Mindestabstandsmatrix zulässige Lösungen gefunden. Obwohl es in diesem Fall keine wahre optimale Lösung gibt, sind die Ergebnisse, die ich in den drei Instanzen scr12, scr15 und scr20 erhalten habe, näher an denselben Instanzen ohne die Mindestabstandsbedingung. Ich kann daher argumentieren, dass meine Lösungen selbst bei Berücksichtigung der Mindestabstandsmatrix näher an den wahren optimalen Lösungen liegen. Bei Instanz tai25b liegt die Optimalgrad meiner Lösung bei mehr als 40 %, was bedeutet, dass mein Pragramm tatsächlich optimieren kann.

Während meiner Untersuchung entdeckte ich die Unzulänglichkeiten des auf Zufallszahlen basierenden Algorithmus. Jedes Mal, wenn ich für die Solver-Klasse einen anderen Seed für den Zufallszahlengenerator einstellte, erhielt ich andere Ergebnisse. Natürlich habe ich mit demselben Seed immer das gleiche Ergebnis erzielt. Allerdings beeinflusst die Wahl des Seed das Endergebnis. Dies wird auch in den Beispielen des Seminars deutlich.

- Es gibt jedoch zwei Einschränkungen.
1. Es gibt keinen einzigen Seed, mit dem man für jede Instanze gleichzeitig die optimale Lösung erreichen kann.
2. es ist nicht möglich, alle Seeds zu durchlaufen, was viel Rechenzeit erfordert

Ich bin daher der Ansicht, dass ich bei meinem Code die Ausgabe des Codes akzeptieren kann, solange die Gesamttransportleistung nach der Optimierung im Vergleich zum nicht optimierten Ergebnis deutlich reduziert wird, während die optimierte Lösung in einer kürzeren Rechenzeit erzielt werden kann.

Die folgenden Codeblöcke sind die Ausgabe der Ergebnisse nach der Anforderungen vom Lehrstuhl.

In [147]:
def outputFormat_in_dict(outputFormat):
    output_dict = {}
    output_dict['ObjVal'] = str(outputFormat.TotalTransport)
    output_dict['Permutation'] = str(list(np.array(outputFormat.Permutation) + 1))

    return output_dict


In [148]:
output_dict_12 = outputFormat_in_dict(outputFormat_12_swap)
print(f'scr12_Lsg_oMA: {output_dict_12}')

output_dict_15 = outputFormat_in_dict(outputFormat_15_swap)
print(f'scr15_Lsg_oMA: {output_dict_15}')

output_dict_20 = outputFormat_in_dict(outputFormat_20_swap)
print(f'scr20_Lsg_oMA: {output_dict_20}')

output_dict_25 = outputFormat_in_dict(outputFormat_25_swap)
print(f'scr25_Lsg_oMA: {output_dict_25}')

scr12_Lsg_oMA: {'ObjVal': '31410', 'Permutation': '[5, 7, 2, 3, 11, 4, 8, 12, 1, 6, 9, 10]'}
scr15_Lsg_oMA: {'ObjVal': '53114', 'Permutation': '[8, 6, 10, 2, 13, 1, 5, 9, 14, 7, 4, 3, 12, 11, 15]'}
scr20_Lsg_oMA: {'ObjVal': '114232', 'Permutation': '[8, 19, 13, 18, 11, 17, 20, 16, 1, 9, 4, 10, 7, 5, 6, 3, 2, 15, 14, 12]'}
scr25_Lsg_oMA: {'ObjVal': '345475531', 'Permutation': '[4, 10, 6, 13, 7, 18, 24, 15, 9, 17, 25, 16, 5, 20, 3, 2, 22, 23, 8, 12, 21, 11, 14, 1, 19]'}


In [149]:
output_dict_12_m = outputFormat_in_dict(outputFormat_12_m_swap)
print(f'scr12_Lsg_mMA: {output_dict_12_m}')

output_dict_15_m = outputFormat_in_dict(outputFormat_15_m_swap)
print(f'scr15_Lsg_mMA: {output_dict_15_m}')

output_dict_20_m = outputFormat_in_dict(outputFormat_20_m_swap)
print(f'scr20_Lsg_mMA: {output_dict_20_m}')

output_dict_25_m = outputFormat_in_dict(outputFormat_25_m)
print(f'scr25_Lsg_mMA: {output_dict_25_m}')

scr12_Lsg_mMA: {'ObjVal': '34556', 'Permutation': '[4, 6, 2, 5, 11, 3, 7, 8, 1, 10, 12, 9]'}
scr15_Lsg_mMA: {'ObjVal': '63128', 'Permutation': '[1, 9, 2, 5, 15, 6, 10, 14, 3, 11, 4, 12, 8, 7, 13]'}
scr20_Lsg_mMA: {'ObjVal': '120878', 'Permutation': '[3, 18, 16, 19, 5, 20, 14, 13, 12, 7, 1, 11, 4, 8, 9, 2, 10, 6, 15, 17]'}
scr25_Lsg_mMA: {'ObjVal': '507556428', 'Permutation': '[25, 18, 7, 4, 5, 6, 9, 3, 19, 10, 11, 17, 13, 14, 15, 16, 21, 2, 8, 20, 23, 22, 12, 24, 1]'}


Wie vom Lehrstuhl gefordert, muss ich einen Solutions-Ordner erstellen, um die generierten json-Dateien zu speichern.

Der folgende Code kann nur einmal ausgeführt werden. Sobald der Ordner Solutions im Dateisystem erstellt wurde, kann er nicht erneut erstellt werden.

In [150]:
import json
import os
path = os.getcwd()
os.mkdir(path+'\\Solutions')
print(path)

d:\6.Semester\4904196_Wanjin Li


In [151]:
# Hier speichere ich alle Szenarien, die die Mindestabstandsmatrix nicht berücksichtigen, in einer json-Datei



output_dict_12_str = json.dumps(output_dict_12, indent=4)
f_12 = open(path +'\\Solutions\\scr12_Lsg_oMA', 'w')
f_12.write(output_dict_12_str)
f_12.close()

output_dict_15_str = json.dumps(output_dict_15, indent=4)
f_15 = open(path + '\\Solutions\\scr15_Lsg_oMA', 'w')
f_15.write(output_dict_15_str)
f_15.close()

output_dict_20_str = json.dumps(output_dict_20, indent=4)
f_20 = open(path + '\\Solutions\\scr20_Lsg_oMA', 'w')
f_20.write(output_dict_20_str)
f_20.close()

output_dict_25_str = json.dumps(output_dict_25, indent=4)
f_25 = open(path + '\\Solutions\\tai25b_Lsg_oMA', 'w')
f_25.write(output_dict_25_str)
f_25.close()

In [152]:
output_dict_12_str_m = json.dumps(output_dict_12_m, indent=4)
f_12_m = open(path + '\\Solutions\\scr12_Lsg_mMA', 'w')
f_12_m.write(output_dict_12_str_m)
f_12_m.close()

output_dict_15_str_m = json.dumps(output_dict_15_m, indent=4)
f_15_m = open(path + '\\Solutions\\scr15_Lsg_mMA', 'w')
f_15_m.write(output_dict_15_str_m)
f_15_m.close()

output_dict_20_str_m = json.dumps(output_dict_20_m, indent=4)
f_20_m = open(path + '\\Solutions\\scr20_Lsg_mMA', 'w')
f_20_m.write(output_dict_20_str_m)
f_20_m.close()

output_dict_25_str_m = json.dumps(output_dict_25_m, indent=4)
f_25_m = open(path + '\\Solutions\\tai25b_Lsg_mMA', 'w')
f_25_m.write(output_dict_25_str_m)
f_25_m.close()